### Import Data

#### - Import Data

In [1]:
# -- Import libraries for HTTP requests, authentication, and URL handling
import requests                           # -- For making HTTP requests to the DHIS2 API
from requests.auth import HTTPBasicAuth   # -- For handling basic authentication in HTTP requests
from urllib.parse import quote            # -- For URL encoding special characters
from getpass import getpass               # -- For securely collecting the passkey (hidden input)

# -- Import data manipulation and numerical libraries
import pandas as pd                       # -- For data manipulation and creating dfs
import numpy as np                        # -- For data manipulation and numerical operations

# -- Import Jupyter Notebook display and interactive utilities
from IPython.display import clear_output  # -- For clearing screen display in Jupyter
from IPython.display import display       # -- For displaying dfs in Jupyter Notebooks
import ipywidgets as widgets              # -- For creating interactive buttons and widgets
from functools import partial             # -- For cleaner argument binding in function calls

# -- Import file, system, and regular expression utilities
import os                                 # -- For operating system interactions (e.g., file paths)
import sys                                # -- For system-specific parameters and functions
import re                                 # -- For working with regular expressions

# -- Import Excel file manipulation and styling tools
from openpyxl import load_workbook        # -- For working with Excel files
from openpyxl.styles import Font, Alignment  # -- For styling and aligning Excel cells

# -- Import document and image processing libraries
from docx import Document                # -- For creating and editing Word documents
from docx.shared import Pt, RGBColor, Inches  # -- For Word document styling (e.g., font size, color, dimensions)
from html2image import Html2Image        # -- For converting HTML to images
from PIL import Image                    # -- For image processing
import pdfkit                            # -- For generating PDFs from HTML

#### - IHVN DHIS2 API

In [2]:
# -- Define the main function to fetch and process DHIS2 data
def fetch_and_process_DHIS2_data(username, password, start_period, end_period, named_urls=None):
    """
    Fetch and process DHIS2 data from named URLs with a user-specified period range, returning a dictionary of processed dfs.
    
    Args:
        username (str): DHIS2 username for authentication
        password (str): DHIS2 password for authentication
        start_period (str): Start period in YYYYMM format (e.g., '202501')
        end_period (str): End period in YYYYMM format (e.g., '202503')
        named_urls (dict, optional): Dictionary where keys are names (e.g., 'Report_Rate_Facility') and values are DHIS2 API URLs.
                                    If None, a default set of URLs is used.
    
    Returns:
        dict: Dictionary where keys are URL names and values are processed dfs
    """
    # -- Step 4: Define separator line
    separator_line = '-' * 43                                 # -- Create a separator line of 43 dashes
    
    # -- Step 1: Define default DHIS2 URLs if none are provided
    if named_urls is None:                                    # -- Check if named_urls is not provided
        named_urls = {                                        # -- Define default DHIS2 URLs
            "Report_Rate_Facility": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3AKH62ia35VIZ%3Bum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AZ7E9RxXmwxG.REPORTING_RATE%3BVmGwLcfPS2N.REPORTING_RATE%3BYFnIy7lATQL.REPORTING_RATE%3BNkuV7xoThHV.REPORTING_RATE%3BHwfLR3npibF.REPORTING_RATE%3BvN9rk5ChByM.REPORTING_RATE%3BoxUN7AXSF8r.REPORTING_RATE&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false",
            "Report_Rate_LGA": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-lmSTo2yxNsA&dimension=dx%3AZ7E9RxXmwxG.REPORTING_RATE%3BVmGwLcfPS2N.REPORTING_RATE%3BYFnIy7lATQL.REPORTING_RATE%3BNkuV7xoThHV.REPORTING_RATE%3BHwfLR3npibF.REPORTING_RATE%3BvN9rk5ChByM.REPORTING_RATE%3BoxUN7AXSF8r.REPORTING_RATE&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "AGYW_MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AQ1KKjeS4seJ%3BzhvcKIWKvEX%3BpgsZWQLbTvw%3BdPefeXOI0MT%3Bb62rwfvBP13%3Bechn4uCHBhF%3BeoSKe92wBYa%3BIYJciZgP6Yt%3BbbSqZH1OAxk%3By5mYZFQbMe6%3BSSa2P2O1keL%3BrtjZkt3ImND%3BDhw4lcmA8i5%3BAMs19im8mm7%3BTNTPcRPC3jV%3BWWaSbjutZof%3BcspEoTIBnOB%3BQ1ntMoY7ZrP%3BjQ51vKvy1SN%3BKj0TobAENyg%3BKRf4sYxv9KG&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "ART_MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AEVJnWv5UQ2I%3BaDFx7U0OSNp%3BuTAVBA24Qgg%3BVNvZaoEcS8M%3BdJpKA2CL66w%3BwUlUHsYzh80%3BmaTBR3htwav%3BN4skK4jJVnm%3BcMpe2pMLJed%3BmJ3Af4Qg0wV%3BxFsrvhyu0Wx%3BCVIR5mDSrr0%3BqO3FSAwqg15%3BE8J56tfMIEa%3BBiqkhMdFIwy%3BfVpRSB8jy9Q%3BudXxeZhT8Fd%3BE20mRpvl5jK%3Bgcg9I4dagWN%3BfBJzc5QIP1b%3BgDuNCzc5liq%3BngCJ4UZOCme%3BEtg6BPVX548%3BiTJ2VvKOWHG%3BrJB5XXrF5zx%3BFRMmrIYSRfz%3BPhUxFwPj2US%3BKtMzH6OTxXL%3Bu0W1SpovSd3%3Bxz9C4uZwMuB%3BLqdahCtUzSX%3Bqb3YzC5X9Lo%3BbFW6JaVxyOo%3BTSzdU77XfK2&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "HTS_MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AsxXB2RokZrt%3BhxYcq0LyXh6%3Bi3I5wU8vm0U%3BwSLnU5ihgb3%3BVz0ZjXCnFtX%3BK7Bkwdae7X1%3BRBxwgKGZYpv%3BjNodLWC4U4d%3Bq6bey1tg06I%3BAZSZngrMzj6%3BpS0Cjik4WEH%3BBrVqJd9MHSA%3BXKu24OQK64R%3BpgiqhOhD95N%3BYL87jqnIzQA%3BmN5TqnUQRgq%3BEvB0bzvxMq0%3BgWVJbVYOT1A%3Bd8lMQuQpLPe%3BT6f8BA7M9Q0%3BSVpx1fVD3bh%3BpJH2bCApdTe%3BuGUbRjQvCeN%3BE3bZgIIG5qQ%3Bd8pnVkpyGlU%3BDn4PJOJJASf%3BQV3wq0WsLXe%3Bs9ksVidbnRG%3BOFj0H56KbKp%3Bh9SMVFhNbBN%3BrJD9ER08iT4%3BJLOnLNuLOva%3BZztEVzPBwwl%3BX6wyczqYmJN%3Bvhhrj8rTq83%3ByR5JEOs2t7q%3BCnSrjlY5Siw%3BUMPLBlGYjWM%3BiE5kXuUJoMC%3BuMqKjodBP76%3Bu9gp0VFbnHh%3BzJuaAQbUlq8%3BQXNyy0dbIpd%3BomaL0XteWO8%3BKtzvesDYABJ%3BHdTQzlvMtjU%3Bqznyc2H90Ay%3Bdtri8UEwZFV%3BFB1EZfVSmYi%3BXzghRlcLMqB%3BOk6fJzfTk75%3BomujB6405jI%3BenPRkAVrHKi%3BE6wddhqJVIN%3BjWajcYCyiDE%3BLSQuUgoTH7o%3Bilp3JWW1qt5%3BURgQ4d3y8WF%3Bxc40kwzTBVG%3BmCD2Qhbd6CY%3BnQtTVOdEMqV%3BKQAEkHXQe7N%3BxBr9Sgyk4cO%3BlI7YzdC7wEd%3BegRRIwWxnJs%3BPPxR0HhCKQR%3BkmBNFDE8duJ%3BGTUzO3HGLWA%3Bh8HFl5EeHHl%3BuXwAyGT9eqW%3BDahOUj6bRk0%3BT8G2KNNZ4eI%3BD7ygy6yCHFs%3BuSQJcqAEvHg%3BU74jSwLCoA1%3BZheiLjTrqRZ%3BjlFslOk3bkU%3BOxBKckVqp29%3BJWHc7D4J132%3Bq1XlBEcB1PE%3BNb95zNUKf29%3BevlfYhoKrjI%3BU8xSE6OneYl%3BF2JEmiJt3Yl%3BPZWrrb7VyCj%3BXxpFcHK1S89%3BsICfdv3or5G%3Bg5HvgkCKSSU%3Bh7XmbTNyTUi%3BUi7DiLwSgqm%3BHIHQHXPOGKl%3Bl3aXhanFimZ%3BSl6d3hqzq2C%3BpIsmPa1GjFs%3BOwrvPMKq0pQ%3BV7hcDYvuPMY%3BDWb8URoPRym%3BL7l30ySaQDy%3BnYrdLtwDlV0%3BifE9LKaLqUm%3BuMkZMIHVVjV%3BcCLFgRUkIww%3BHUWYU2ruloN%3BoXsckqTpzhN%3Bm4CzDuc50Jn%3BLPyxdv2eBEj%3BZLARI5LYBOL%3BPYL5GdQGPfI%3BmQZ4z94jzak%3BRkwT0w0mPNj%3BEC5iN37lZ61%3BS0QI1IcESjq%3BnR3ZklnEuBy%3BgxZKoTwyLnn%3BIp75Jb7o8Au%3BVPMZR303TWP%3BjqP0tN3v5hJ%3BuoBeVy413Mj%3BPn1GooQL0I4&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "HTS_MSF_HIVST_approach": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AT8G2KNNZ4eI&dimension=tBdRxXi3Dxr%3AALL_ITEMS&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "NSP_MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AxesSKTdzhPF%3BuIx4sChOeMC%3BkJSLUKrBKez%3BJylie7CPg63%3BTV5DhhOgF7s%3BL0X8cbiEfrC%3BoSDqDrfRpb3%3BIQYpNyC6lF2%3BlOVVI8B4Ag6%3BRxQv56EIuWs%3BqyV24hEIgM5%3BObJbRGQ3QPI%3BptgY5CK3GUc%3BC16TIH6Zxju%3BKY2gb90cvTu%3BO3g2Lq9fpUU%3Bkk3hEztWa48%3BxvfqoSSOIOL%3BcLSlAzBug6Y%3BayTk1t8sjFN%3BjY2SLdSlMug%3Bv3sfwf9O1R9%3BxrHzg7SORIt&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "PMTCT_MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202502&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3ABWLsXE490Yw%3BAtzuY8wFIaP%3BmjoPYWLowE6%3BhWkEco9hG0R%3BgybVHn9Y3SQ%3BkNEHSaIxOja%3BSWsoIgeQBKd%3Bym1WjkXu8ol%3BHDVcSJrN68V%3BIY3zDXL1t0b%3BCrv4RYmAJot%3Bn1MZAkrNWlM%3BGDFB54Uuepj%3BPaXizDbvXcE%3BhnqKzrB2hX0%3BZv0Go6hoDxZ%3BZOzgBnzWLOK%3Bbj8jc0HujI2%3BVlhKdJD8Mav%3BqO3KHhkVM38%3BkSHVwcO9iYZ%3BWnEhg2SiMjF%3BFR9ibOgyNKg%3BbyiZy4Xo7Rx%3BvpEeS23uOzB%3BzoSht7Xk3OI%3BUMFtzIDX9nE%3BxkQqd3C5vd8%3BCAO8uuth3q6%3BRATzsmd7JfL%3BDPFGLfkzZTF%3BZZEXn1RzLwL%3BuMATgKm1ZEV%3ByShXhGc00TY%3Bdc0p7yc1jPy%3BJsyAmKQMqK1%3BlIJwQ20CgPE%3BbxeHOqH0s2V%3BOum5ofYWexC%3BadqiOXDSRbU%3BHFq8NihMhgw%3BcibdW4TLOJ5%3BMJHhWCCOy1q%3BPQHBZmMVWE3%3Bw9NSJbXp5Ub%3BCyrYJsUOe7Q%3BER9cFdvluIM%3BUP3mRp6Zr69%3Bgl5kyYu85zX%3BrIY8zpZJxtv%3BaPtwtDwhOeL%3BuQEZL5j8ZHS%3BSpuoRhODIuZ%3Bvu5TP1CuesM%3BeZ4kUaPlQWC%3BSVDAcLT6IKS%3BFLt9JLhBWlK%3Bt9NJ4JhsXVl%3BnOpah0JpIzI%3BDuFJekp1ymh%3BTJ5BREHjZnu%3BWzAmqotjxV7%3BwRRoicp2S4Y%3Bzq6Tvgk5GFl%3BLMrI0jXhjP4%3BTDLVNVgHUvf%3BakVDx9ew0y8%3ByDAbvAlPR69%3Bi6rCuewXarq%3BzKYuhIKhMKM%3BGQ4kUNGMf7i%3Bv2e5LaziiBr%3BsLWmlnDQzgt%3BrKzvhBgkpe5%3BpyUW87YFbY5%3BssvxYtnGXpp%3BlY6wzD8718l%3BeLZuFwiv4rQ%3BDRFLRwv7f8F%3BLgbHzzQILZY%3BWt8jhQxqq9L%3BTdqRbqoS3Uj%3BcfUpkarx8og%3BWbRBSkKaTE8%3BEvZz6jNSRMX%3BYGZr2K5Y0Yq%3BJGj4bfhtzCk%3BkKSgWbeTAL1%3Bo9zPtZI4c9T%3BcmkwuM6ZOaN%3BcGPBuUrZ5Oi%3BN4WysYNBRFk%3Bd0izXgJhn2P%3BxUYwPZH7ulJ%3BykZcKOVoJeH%3Bj3ycCFq6vzR%3BK2lEmtE4xjz%3Bw4jlyeHkVTD%3BDbj5r3zevDF&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "KP_Prev_MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3ArJFps9PGgYI%3BpFrm4E6yE0i%3Bwld8ChYjUUS%3Bhnqs8zD2RBz%3BFdogfvZ9Es5%3BUWnjd2hDMpZ%3BmM3TZYEkMaF%3BK93mvkTXpbl%3BtIZvjax5TD5%3BN05zqacXFsD%3BcVHdvMKuwse%3BBFSM6D4KWg4%3ByHBgGkS0dPr%3BEw3ZRxdYY7g%3BihRnS748lOC%3BlIu2vmR2zHU%3BfbxvYGgVsrK%3BTdATue0b5th%3BFBBKTzFqBVR%3BF27CfSgCmSk%3BdecdNlpZy4B%3BRVuw3Ny3ysw%3BhN5ZT3zziqA%3BKngVqiOqAdp%3Bdfznt6rhrYg%3BbxK7Uh32HMK%3BduzVXts1QUh%3BXpDOj1a7ZdC%3BogXBJLCw1Qe%3BAKQTKPoUHt6%3BrClFr1FdYVY%3BdTWbSB6HZg6%3BGZo5a5CwzV7%3BdIElcrfEgSB%3BP8oCCg2mUcG%3BZc7dmnNEQwj%3BmWQsKiSyv7J%3BQVOL72RTOZR%3Bri9iXOK3wxX%3Bpqp8Tyvsdc1%3BNat3LtmEWXD%3BPm78aQ5UVh8%3BWaI8nE16Ab0%3BfHlNG1CMqYX%3BGwxEq7jrcSc%3BiqqfsLcdWiK%3BeF7Yi0cLTXP%3BVnDrAREzUE2%3BSSxprjJB8vd%3BL5pAPbPCrNF%3BPrdKtyHsOE8%3BLNQtCiSCfnC%3Bb3JIlAZUjNJ%3BeK3YlBsHGkB%3BTz9WrAjxVY9%3BMkl91koS5cn%3BdAQlguHfLF8%3BBdgtqOw1P33%3BhZzpfxOQo4b%3BJBoik13RWdP%3BhAVQ3sTq3nJ%3BGKiJWrIeEJu%3BqqFvQly7I9h%3BJtOlLSk4yTF%3BUfMiT3LdXYc%3BdLj7PgakBi5%3BUaOdcjrcI59%3BssQZNWweFrL%3BhHkzZLsAqmI%3BkYAhnVk8fYp%3BkTIGrSkqbrG%3Bl4ZD3aaxH7j%3BWAZm2HXm3xI%3BSECmJH5Lfer%3BKHnHX7Rb29d%3BVE0Z777wXB2%3Bzgqte2mbKxT%3BDbavrmvoWPG%3BJgDiY0dv2RX%3BS8WFXWTFvai%3BBBwzjs2JQ1Z%3BxRJrWUdpAxm%3BYS8QAyBakNp%3BJfPSsLKaeDV%3BkmsvoBtZ4oa&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID"
        }

    # -- Step 2: Extract dx IDs from named_urls
    all_dx_ids = set()                                        # -- Initialize empty set to store unique dx IDs
    for url in named_urls.values():                           # -- Iterate over each URL in named_urls
        dx_matches = re.findall(r'dx%3A([^&]+)', url)         # -- Extract dx parameter from URL using regex
        for match in dx_matches:                              # -- Process each dx match found
            ids = [id.split('.')[0] for id in match.split('%3B')]  # -- Split by %3B and remove .REPORTING_RATE suffixes
            all_dx_ids.update(ids)                            # -- Add cleaned IDs to the set

    # -- Step 3: Fetch only relevant data element descriptions
    base_url = 'https://ihvn.dhistance.com/api/dataElements'  # -- Set base URL for data elements endpoint
    dx_filter = f"id:in:[{','.join(all_dx_ids)}]"             # -- Create filter string for specific dx IDs
    params = {                                                # -- Define query parameters for the API request
        'fields': 'id,name,description',                      # -- Request id, name, and description fields
        'filter': dx_filter,                                  # -- Apply filter for specific IDs
        'paging': 'false'                                     # -- Disable pagination to get all results in one response
    }
    try:
        response = requests.get(base_url, auth=HTTPBasicAuth(username, password), params=params)
        response.raise_for_status()
        data_elements = response.json().get('dataElements', [])
        print(f"Fetched {len(data_elements)} data elements")
        dataelement_to_description = {de['id']: de.get('description', de['name']) for de in data_elements}
    except requests.exceptions.HTTPError as e:
        if response.status_code == 401:
            print(separator_line)
            print(f"⦸ Error: Failed to fetch descriptions: IHVN DHIS2 login credentials invalid")
            print(separator_line)
            return
        print(f"⦸ Error: Failed to fetch descriptions: IHVN DHIS2 login credentials invalid")
        dataelement_to_description = {}
    except requests.exceptions.RequestException as e:
        print(f"⦸ Error: Failed to fetch descriptions: IHVN DHIS2 login credentials invalid")
        dataelement_to_description = {}

    # -- Step 5: Validate and parse the start and end periods
    # -- Step 5.1: Validate start and end periods
    for period in [start_period, end_period]:                 # -- Check both start and end periods
        if not (isinstance(period, str) and len(period) == 6 and period.isdigit() and 1 <= int(period[4:]) <= 12):  # -- Validate format and month range
            print(separator_line)                             # -- Print separator line for readability
            print("⦸ Error: Invalid period date format")      # -- Print error message
            print(separator_line)                             # -- Print separator line for readability
            return {}                                         # -- Return empty dict to halt execution

    # -- Step 5.2: Parse the start and end periods
    start_year = int(start_period[:4])                        # -- Extract year from start period (e.g., 2025)
    start_month = int(start_period[4:])                       # -- Extract month from start period (e.g., 01)
    end_year = int(end_period[:4])                            # -- Extract year from end period (e.g., 2025)
    end_month = int(end_period[4:])                           # -- Extract month from end period (e.g., 03)

    # -- Step 6: Format the period range for display
    month_names = {                                           # -- Define month names for display
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
        7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    }
    start_display = f"{month_names[start_month]}{str(start_year)[-2:]}"  # -- Format start period (e.g., Jan-25)
    end_display = f"{month_names[end_month]}{str(end_year)[-2:]}"        # -- Format end period (e.g., Mar-25)

    # -- Step 7: Print separator line to highlight successful setup
    print('Data processed and stored as:')                    # -- Print processing start message
    print(separator_line)                                     # -- Print separator line after message

    # -- Step 8: Generate a list of periods between start and end
    periods = []                                              # -- Initialize empty list for periods
    current_year, current_month = start_year, start_month     # -- Set starting point for period generation
    while (current_year < end_year) or (current_year == end_year and current_month <= end_month):  # -- Loop until end period is reached
        periods.append(f"{current_year}{current_month:02d}")  # -- Add period in YYYYMM format (e.g., 202501)
        current_month += 1                                    # -- Increment month
        if current_month > 12:                                # -- If month exceeds 12
            current_month = 1                                 # -- Reset to January
            current_year += 1                                 # -- Increment year

    # -- Step 9: Encode the periods for URL use
    period_string = "%3B".join(periods)                       # -- Join periods with %3B (URL-encoded semicolon)
    period_param = f"dimension=pe%3A{period_string}"          # -- Format period parameter for API URL

    # -- Step 10: Initialize data storage and counters
    processed_data = {}                                       # -- Initialize dict to store processed DataFrames
    success_count = 0                                         # -- Counter for successful URL processes
    total_urls = len(named_urls)                              # -- Total number of URLs to process

    # -- Step 11: Define cluster mapping for LGAs
    cluster = {                                               # -- Define mapping of LGAs to clusters
        "an Aguata": "Aguata", "an Anaocha": "Aguata", "an Orumba North": "Aguata", "an Orumba South": "Aguata",
        "an Awka North": "Awka", "an Awka South": "Awka", "an Dunukofia": "Awka", "an Idemili North": "Awka",
        "an Idemili South": "Awka", "an Njikoka": "Awka", "an Ekwusigo": "Nnewi", "an Ihiala": "Nnewi",
        "an Nnewi North": "Nnewi", "an Nnewi South": "Nnewi", "an Anambra East": "Omambala",
        "an Anambra West": "Omambala", "an Ayamelum": "Omambala", "an Oyi": "Omambala", "an Ogbaru": "Onitsha",
        "an Onitsha North": "Onitsha", "an Onitsha South": "Onitsha"
    }

    # -- Step 12: Process each named URL
    for url_name, url in named_urls.items():                  # -- Iterate over each name-URL pair
        # -- Step 12.1: Update URL with new period range
        if "dimension=pe%3A" in url:                          # -- Check if URL has a period dimension
            start_idx = url.find("dimension=pe%3A")           # -- Find start of period parameter
            end_idx = url.find("&", start_idx) if url.find("&", start_idx) != -1 else len(url)  # -- Find end of period parameter
            url = url[:start_idx] + period_param + url[end_idx:]  # -- Replace old period with new one
        else:                                                 # -- If no period dimension exists
            url = url + "&" + period_param if "?" in url else url + "?" + period_param  # -- Append period parameter

        # -- Step 12.2: Fetch data from DHIS2 API
        try:
            response = requests.get(url, auth=HTTPBasicAuth(username, password))  # -- Send GET request with authentication
            response.raise_for_status()                       # -- Raise exception if request fails
            data = response.json()                            # -- Parse response into JSON
        except requests.exceptions.RequestException as e:     # -- Catch request-related exceptions
            print(f"⦸ Error: No signal to get '{url_name}' data")  # -- Print error message
            continue                                          # -- Skip to next URL

        # -- Step 12.3: Extract table structure from JSON
        headers = [header['name'] for header in data.get('headers', [])]  # -- Get column names from headers
        df = pd.DataFrame(data.get('rows', []), columns=headers)  # -- Create DataFrame from rows
        df = df.rename(columns={'dx': 'dataElement', 'ou': 'orgUnit', 'pe': 'period'})  # -- Standardize column names

        # -- Step 12.4: Extract metadata from the JSON response
        meta = data.get('metaData', {})                       # -- Get metadata section
        ou_hierarchy = meta.get('ouHierarchy', {})            # -- Get organizational unit hierarchy
        items = meta.get('items', {})                         # -- Get item mappings (IDs to names)

        # -- Step 12.5: Create organizational unit mappings based on URL name
        if url_name == 'Report_Rate_LGA':                     # -- Special case for Report_Rate_LGA
            orgunit_to_level = {ou: ou for ou in df['orgUnit'].unique()}  # -- Map orgUnit to itself
        else:                                                 # -- For other URLs
            orgunit_to_level = {                              # -- Extract second level from hierarchy
                ou: ou_hierarchy.get(ou, '').split('/')[1] if '/' in ou_hierarchy.get(ou, '') else ou
                for ou in df['orgUnit'].unique()              # -- Iterate over unique orgUnits
            }

        # -- Step 12.6:  Create name mappings for organizational units and data elements
        level_to_name = {                                     # -- Map orgUnit level IDs to names
            org_id: items[org_id]['name']
            for org_id in set(orgunit_to_level.values()) if org_id in items  # -- Only include IDs in items
        }
        orgunit_to_name = {                                   # -- Map orgUnit IDs to names
            ou: items[ou]['name']
            for ou in df['orgUnit'].unique() if ou in items   # -- Only include orgUnits in items
        }
        dataelement_to_name = {                               # -- Map dataElement IDs to their names
            de: items[de]['name']
            for de in df['dataElement'].unique() 
            if de in items                                    # -- Only include dataElements present in items
        }

        # -- Step 12.7: Pivot the df to reshape the data
        if url_name == 'HTS_MSF_HIVST_approach':              # -- Special case for HTS_MSF_HIVST_approach
            pivoted_df = df.pivot(                            # -- Pivot using tBdRxXi3Dxr column
                index=['period', 'orgUnit'],                  # -- Set index columns
                columns=['tBdRxXi3Dxr'],                      # -- Set pivot column
                values='value'                                # -- Set value column
            ).reset_index()                                   # -- Reset index to columns
        else:                                                 # -- For other URLs
            pivoted_df = df.pivot(                            # -- Pivot using dataElement column
                index=['period', 'orgUnit'],                  # -- Set index columns
                columns='dataElement',                        # -- Set pivot column
                values='value'                                # -- Set value column
            ).reset_index()                                   # -- Reset index to columns
        pivoted_df.columns.name = None                        # -- Clear column index name

        # -- Step 12.8: Format the 'period' column to 'Mon-YY' (e.g., Jan-24)
        pivoted_df['period'] = pd.to_datetime(pivoted_df['period'], format='%Y%m').dt.strftime('%b-%y')  # -- Convert YYYYMM to Mon-YY

        # -- Step 12.9: Add organizational hierarchy information to the pivoted df
        pivoted_df['orgunitlevel'] = pivoted_df['orgUnit'].map(orgunit_to_level)  # -- Add orgUnit level
        pivoted_df['LGA'] = pivoted_df['orgunitlevel'].map(level_to_name)         # -- Add LGA name
        pivoted_df['orgUnit'] = pivoted_df['orgUnit'].map(orgunit_to_name)        # -- Replace orgUnit ID with name
        pivoted_df['Cluster'] = pivoted_df['LGA'].map(cluster)                    # -- Add cluster mapping

        # -- Step 12.10: Rename columns with dataelemenet descriptions
        if url_name in ['Report_Rate_Facility', 'Report_Rate_LGA']:
            rename_dict = {                                   # -- Create a dictionary for renaming columns
                **{col: dataelement_to_name[col] for col in pivoted_df.columns if col in dataelement_to_name},  # -- Rename dataElement columns to their names
                'period': 'ReportPeriod',                         # -- Rename 'period' to 'ReportPeriod'
                'orgUnit': 'FacilityName'                         # -- Rename 'orgUnit' to 'FacilityName'
            }
        else:
            rename_dict = {                                       # -- Create renaming dictionary
                **{col: dataelement_to_description.get(col, col) 
                   for col in pivoted_df.columns if col in dataelement_to_description},  # -- Use descriptions for data elements
                'period': 'ReportPeriod',                         # -- Rename period to ReportPeriod
                'orgUnit': 'FacilityName'                         # -- Rename orgUnit to FacilityName
            }
        pivoted_df.rename(columns=rename_dict, inplace=True)      # -- Apply renaming

        # -- Step 12.11: Handle NaN values
        if url_name in ['Report_Rate_Facility', 'Report_Rate_LGA']:  # -- Check if URL is a report rate type
            pivoted_df.fillna('', inplace=True)                   # -- Replace NaN with empty string
        else:                                                     # -- For other URLs
            pivoted_df.fillna(0, inplace=True)                    # -- Replace NaN with 0

        # -- Step 12.12: Finalize DataFrame
        pivoted_df = pivoted_df.reset_index(drop=True)            # -- Reset index and drop it
        processed_data[url_name] = pivoted_df                     # -- Store processed DataFrame
        success_count += 1                                        # -- Increment success counter
        print(f"- {url_name}")                                    # -- Print URL name as processed

    # -- Step 13: Display processing summary
    global report_period_display
    report_period_display = (                                 # -- Format summary message
        f"✔️ Data fetched successfully!\nReport extracts: ({success_count}/{total_urls})\n"
        f"Extraction period: {start_display} - {end_display}\nWorkbook variables & functions loaded"
    )
    if processed_data:                                        # -- Check if any data was processed
        print(separator_line)                                 # -- Print separator line before message
        print(report_period_display)                          # -- Print success message
        print(separator_line)                                 # -- Print separator line after message
    else:                                                     # -- If no data was processed
        print(separator_line)                                 # -- Print separator line before message
        print(f"⦸ Failed:\nReport extracts: (0/{total_urls})\nIHVN DHIS2 login credentials invalid")  # -- Print failure message
        print(separator_line)                                 # -- Print separator line after message

    # -- Step 14: Return processed data
    return processed_data                                     # -- Return dictionary of processed DataFrames

#### - Functions: Variables

In [3]:
def load_variables():
    """
    Defines variables for the notebook and assigns them as global.
    """
    # Local imports to ensure dependencies are met
    import os
    
    # Declare all variables as global
    global file_path, report_name, report_name_rate, report_name_outlier, report_name_period
    global report_name_period_name, report_period_name_folder, sub_folder_image_file, sub_folder_doc_file
    global sub_folder2_image_file_report_rate, sub_folder2_image_file_msf_outlier
    global doc_file_report_rate_xlsx, doc_file_msf_outlier_docx, doc_file_msf_outlier_xlsx
    global highlight_red_list, MSF_hierarchy, MSF_report_rate_columns    

    try:
        # -- Step 1: Define main report export path
        try:
            file_path = r'C:\Users\HP\Desktop\ANSO Nnewi\CQI\python\report\msf\ihvn'
        except Exception as e:
            print(f"⦸ Error defining file_path: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 2: Create report name, dynamic period, and joined report period name
        try:
            report_name = "ANSO MSF report"
            report_name_rate = "ANSO MSF report rate"
            report_name_outlier = "ANSO MSF outlier"
            report_name_period = DHIS2_data['Report_Rate_Facility'].ReportPeriod.iloc[0]
            report_name_period_name = f"{report_name_period} {report_name}"
        except Exception as e:
            print(f"⦸ Error defining report names or accessing DHIS2_data: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 3: Create report period folder
        try:
            report_period_name_folder = os.path.join(file_path, f"{report_name_period_name}")
        except Exception as e:
            print(f"⦸ Error creating report_period_name_folder: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 4: Define folders for storing reports
        try:
            sub_folder_image_file = os.path.join(report_period_name_folder, "image file")
            sub_folder_doc_file = os.path.join(report_period_name_folder, "document file")
        except Exception as e:
            print(f"⦸ Error defining sub_folder_image_file or sub_folder_doc_file: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 4.1: Add subfolders for specific report types
        try:
            sub_folder2_image_file_report_rate = os.path.join(sub_folder_image_file, f"{report_name_rate}")
            sub_folder2_image_file_msf_outlier = os.path.join(sub_folder_image_file, f"{report_name_outlier}")
        except Exception as e:
            print(f"⦸ Error defining sub_folder2_image_file_report_rate or sub_folder2_image_file_msf_outlier: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 5: Create folders if they do not exist
        try:
            os.makedirs(sub_folder_image_file, exist_ok=True)
            os.makedirs(sub_folder_doc_file, exist_ok=True)
            os.makedirs(sub_folder2_image_file_report_rate, exist_ok=True)
            os.makedirs(sub_folder2_image_file_msf_outlier, exist_ok=True)
        except Exception as e:
            print(f"⦸ Error creating directories: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 6: Define report document file paths
        try:
            doc_file_report_rate_xlsx = os.path.join(sub_folder_doc_file, f"{report_name_period} {report_name_rate}.xlsx")
            doc_file_msf_outlier_docx = os.path.join(sub_folder_doc_file, f"{report_name_period_name}.docx")
            doc_file_msf_outlier_xlsx = os.path.join(sub_folder_doc_file, f"{report_name_period_name}.xlsx")
        except Exception as e:
            print(f"⦸ Error defining document file paths: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 7: Define list of words and phrases to keep after HTML cleaning
        try:
            highlight_red_list = [
                'Community', 'Walk-In', 'Community & Walk-In', 'subset of 4',
                'Self, Spouse, Sexual Partner, Children, Social Network, Others',
                'FSW, MSM, PWID, TG, Others', 'Testing frequency', 'Outreach',
                'Outreach-Pregnant', 'Outreach-Others', 'Excluding community testing',
                'Excluding previously known', 'IPV', 'ANC', 'L&D', '<72hrs PP',
                '<72 hrs', '>72 hrs - < 6 months', '>6 - 12 months',
                'ANC, L&D, <72hrs Post Partum', 'Facility', 'Outside Facility',
                'Within and outside the facility', 'within 72 hrs of birth',
                'between >72 hrs - <2 months of birth', 'All regimens', 'Regimen Lines',
                'MMD', 'DSD', 'excludes ART transfer-in', 'ART Addendum-2'
            ]
        except Exception as e:
            print(f"⦸ Error defining highlight_red_list: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 8: Define MSF hierarchy
        try:
            MSF_hierarchy = ['ReportPeriod', 'Cluster', 'LGA', 'FacilityName']
        except Exception as e:
            print(f"⦸ Error defining MSF_hierarchy: {str(e)}")
            raise  # Re-raise to trigger top-level except

    except Exception as e:
        print(f"⦸ Error loading variables: {str(e)}")
        # Optionally assign fallback values to globals, but here we just return
        return

#### - Function: Validations & Exports

In [4]:
def load_functions():
    """
    Defines and assigns global functions for styling dfs and exporting them to image, Excel, and Word formats.
    """
    try:
        # -- Step 1: Declare global functions and variables
        global outlier_red, outlier_green, export_df_to_doc_image_excel  # -- Declare styling and export functions as global
        global filter_gap_and_check_empty_df, prepare_and_convert_df     # -- Declare DataFrame processing functions as global
        global Pre_MSF_positives_all

        # -- Step 2: Define outlier_red function
        # -- Function: Style cells with light coral for values less than 100
        def outlier_red(val):                                           # -- Define function to style cells red
            """
            Styles cells with a light coral background and bold font for values less than 100.
            Applies a border for consistent formatting.
    
            Args:
                val: Value to evaluate (int, float, or string).
    
            Returns:
                str: CSS style string if condition met.
            """
            try:                                                        # -- Begin try block for error handling
                # -- Step 2.1: Define conditions for styling
                condition = (                                           # -- Combine all conditions for red styling
                    ((isinstance(val, (int, float)) and val < 100) or   # -- Check if numeric and less than 100
                     (isinstance(val, object) and val != '100' and val != ''))  # -- Check if string, not '100', and not empty
                    or                                                  # -- OR condition for additional cases
                    (not (isinstance(val, (int, float)) and val < 100) and  # -- Check if not numeric less than 100
                     (isinstance(val, (int, float)) and val != 0))      # -- AND numeric not equal to 0
                ) and (not (isinstance(val, (int, float)) and val == 0))   # -- AND ensure exclusion of numeric 0

                # -- Step 2.2: Apply styling if condition is met
                if condition:                                           # -- Evaluate combined condition
                    return 'background-color: lightcoral; font-weight: bold; border-bottom: 0.01px solid #f3f3f3;'  # -- Return red styling for matching values
                return None                                             # -- Return None if no styling applies
            except Exception as e:                                      # -- Catch exceptions in outlier_red definition
                print(f"⦸ Error defining outlier_red: {str(e)}")        # -- Print error message
                return None                                             # -- Return None on error
        
        # -- Step 3: Define outlier_green function
        # -- Function: Style cells with light green for values equal to 100
        try:
            def outlier_green(val):
                """
                Styles cells with a light green background and bold font for values equal to 100.
                Applies a border for consistent formatting.
                
                Args:
                    val: Value to evaluate (int, float, or string).
                
                Returns:
                    str: CSS style string if condition met, None otherwise.
                """
                if (isinstance(val, (int, float)) and val == 100) or (isinstance(val, object) and val == '100' and val != ''):
                    return 'background-color: lightgreen; font-weight: bold; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_green: {str(e)}")           # -- Print error message for outlier_green
            outlier_green = lambda val: None                            # -- Fallback to a no-op function

        # -- Step 4: Define export_df_to_doc_image_excel function
        try:
            def export_df_to_doc_image_excel(
                report_name=None,
                df_shape=None,
                df_style=None,
                img_file_name=None,
                img_file_path=None,
                doc_description=None,
                doc_indicators_to_italicize=None,
                doc_indicators_to_underline=None,
                doc_file_path=doc_file_msf_outlier_docx,
                xlm_file_path=None,
                xlm_sheet_name=None,
                highlight_red_list=highlight_red_list
            ):
                """
                Exports a df to an image, Excel file, and Word document with optional description for the document.
                (Docstring truncated for brevity; see original for full details.)
                """
                try:
                    # -- Step 4.1: Process image export
                    if all([df_shape is not None, df_style is not None, img_file_name is not None, img_file_path is not None]):  # -- Check if all image params are provided
                        n_rows, n_cols = df_shape.shape                 # -- Get DataFrame dimensions
                        fig_width = max(10, n_cols * 1.5)               # -- Calculate figure width based on column count
                        fig_height = max(5, n_rows * 1)                 # -- Calculate figure height based on row count

                        styled_df = (df_style.set_table_styles([        # -- Apply table styles to DataFrame
                            {'selector': 'th', 'props': [('max-width', '100px'), ('word-wrap', 'break-word'), ('text-align', 'right'), ('background-color', 'white'), ('padding', '2px 10px')]},
                            {'selector': 'td', 'props': [('text-align', 'right'), ('padding', '2px 10px')]},
                            {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
                            {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white')]}
                        ]).set_table_attributes('style="font-family: Calibri; font-size: 12pt; border-collapse: collapse;"'))

                        html = f"""
                        <div style="display: flex; justify-content: center; align-items: center; height: 100vh;">
                            {styled_df.to_html()}
                        </div>
                        """

                        hti = Html2Image()                              # -- Initialize Html2Image for screenshot
                        hti.output_path = img_file_path                 # -- Set output path for image
                        image_path = os.path.join(img_file_path, img_file_name)  # -- Define full image path
                        hti.screenshot(html_str=html, save_as=img_file_name, size=(int(fig_width * 100), int(fig_height * 100)))  # -- Generate screenshot

                        img = Image.open(image_path).convert('RGBA')    # -- Open image for cropping
                        bbox = img.getbbox()                            # -- Get bounding box for cropping
                        if bbox:                                        # -- Check if bounding box exists
                            cropped_img = img.crop(bbox)                # -- Crop image to content
                            cropped_img.save(image_path, format='PNG')  # -- Save cropped image
                    else:
                        image_path = None                               # -- Set image_path to None if params missing

                    # -- Step 4.2: Process Excel export
                    if all([df_style is not None, xlm_file_path is not None, xlm_sheet_name is not None]):  # -- Check if all Excel params are provided
                        xlm_sheet_name = xlm_sheet_name[:31]            # -- Truncate sheet name to 31 characters
                        mode = 'a' if os.path.exists(xlm_file_path) else 'w'  # -- Set mode: append if file exists, write if not
                        with pd.ExcelWriter(xlm_file_path, engine='openpyxl', mode=mode, if_sheet_exists='replace' if mode == 'a' else None) as writer:  # -- Open Excel writer
                            df_style.to_excel(writer, sheet_name=xlm_sheet_name, index=False)  # -- Write styled DataFrame to Excel

                        wb = load_workbook(xlm_file_path)               # -- Load workbook for further formatting
                        ws = wb[xlm_sheet_name]                         # -- Select worksheet by truncated name

                        for cell in ws[1]:                              # -- Process header row for formatting
                            if cell.value and isinstance(cell.value, str):  # -- Check if cell has string value
                                protected_map = {}                      # -- Initialize map for protected phrases
                                for phrase in highlight_red_list or []: # -- Iterate over phrases to highlight
                                    token = f"@@PROTECT_{abs(hash(phrase))}@@"  # -- Create unique token
                                    protected_map[token] = phrase       # -- Map token to phrase
                                    cell.value = cell.value.replace(phrase, token)  # -- Replace phrase with token
                                cell.value = re.sub(r"<.*?>", "", cell.value)  # -- Remove HTML tags
                                for token, phrase in protected_map.items():  # -- Restore protected phrases
                                    cell.value = cell.value.replace(token, phrase)  # -- Replace token with phrase
                                cell.value = cell.value.strip()         # -- Strip whitespace

                        font_style = Font(name='Calibri', size=8)       # -- Define font style for cells
                        header_font = Font(name='Calibri', size=8, bold=True)  # -- Define font style for headers
                        header_alignment = Alignment(horizontal="left", vertical="bottom", wrap_text=True)  # -- Define header alignment

                        for row in ws.iter_rows():                      # -- Apply font style to all cells
                            for cell in row:
                                cell.font = font_style

                        for cell in ws[1]:                              # -- Apply header formatting
                            cell.alignment = header_alignment
                            cell.font = header_font

                        for col in ws.iter_cols(min_col=1, max_col=4):  # -- Adjust column widths
                            max_length = max((len(str(cell.value)) if cell.value else 0) for cell in col)  # -- Find max length
                            ws.column_dimensions[col[0].column_letter].width = max_length  # -- Set column width

                        ws.auto_filter.ref = ws.dimensions              # -- Enable auto-filter for sheet
                        wb.save(xlm_file_path)                          # -- Save workbook

                    # -- Step 4.3: Create or append to Word document
                    if all([doc_file_path is not None, image_path is not None, doc_indicators_to_italicize is not None, doc_indicators_to_underline is not None]):  # -- Check if all doc params are provided
                        if os.path.exists(doc_file_path):               # -- Check if document exists
                            doc = Document(doc_file_path)               # -- Load existing document
                        else:
                            doc = Document()                            # -- Create new document

                        style = doc.styles['Normal']                    # -- Set default style
                        style.font.name = 'Calibri'                     # -- Set font to Calibri
                        style.font.size = Pt(9.5)                       # -- Set font size

                        for section in doc.sections:                    # -- Configure section margins
                            section.left_margin = Inches(0.5)
                            section.right_margin = Inches(0.5)
                            section.top_margin = Inches(1)
                            section.bottom_margin = Inches(1)

                        if doc_description:                             # -- Add description if provided
                            title_paragraph = doc.add_heading(report_name, level=2)  # -- Add report name as heading
                            title_run = title_paragraph.runs[0]         # -- Get title run
                            title_run.font.size = Pt(10)                # -- Set title font size
                            title_run.font.color.rgb = RGBColor(0, 0, 0)  # -- Set title color

                            paragraph = doc.add_paragraph()             # -- Add paragraph for description
                            paragraph.paragraph_format.space_after = Pt(0)  # -- Remove space after paragraph

                            phrases_to_bold = [                         # -- Define phrases to bold
                                "REPORT ONLY 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY HEI ARVs FOR 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY EID SAMPLE COLLECTION FOR 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY EID PCR RESULTS FOR 2025 LIVE BIRTHS BY PPW",
                                "Report Name:", "should not be greater than",
                                "should not be lesser than", "should not be equal to",
                                "should be greater than", "should be lesser than",
                                "should be equal to", "plus", "Note", "OR"
                            ]
                            all_phrases = phrases_to_bold + (doc_indicators_to_italicize or []) + (doc_indicators_to_underline or [])  # -- Combine all phrases
                            pattern = r'|'.join(re.escape(phrase) for phrase in all_phrases)  # -- Create regex pattern
                            matches = list(re.finditer(pattern, doc_description))  # -- Find matches in description

                            last_index = 0                              # -- Track last processed index
                            for match in matches:                       # -- Process each match
                                start, end = match.start(), match.end() # -- Get match boundaries
                                paragraph.add_run(doc_description[last_index:start])  # -- Add text before match
                                run = paragraph.add_run(doc_description[start:end])  # -- Add matched text
                                if match.group(0) in phrases_to_bold:   # -- Apply bold if in bold list
                                    run.bold = True
                                if match.group(0) in doc_indicators_to_italicize:  # -- Apply italic if in italicize list
                                    run.italic = True
                                if match.group(0) in doc_indicators_to_underline:  # -- Apply underline if in underline list
                                    run.underline = True
                                last_index = end                            # -- Update last index

                            paragraph.add_run(doc_description[last_index:])  # -- Add remaining text

                        doc.add_picture(image_path, width=Inches(7))    # -- Add image to document

                        section = doc.sections[-1]                      # -- Get last section for footer
                        footer = section.footer.paragraphs[0]           # -- Access footer paragraph
                        footer.text = "This is an auto-generated report. Ensure all data is reviewed before any update is made."  # -- Set footer text
                        footer.runs[0].font.size = Pt(7.5)             # -- Set footer font size
                        footer.runs[0].font.color.rgb = RGBColor(100, 100, 100)  # -- Set footer color

                        doc.save(doc_file_path)                         # -- Save document

                    # -- Step 4.4: Generate and print success messages
                    if all([report_name, img_file_name, img_file_path, xlm_file_path, xlm_sheet_name]):  # -- Check if all params for success message are provided
                        img_file_path_name = os.path.basename(img_file_path)  # -- Get base image path name
                        xlm_file_path_name = os.path.basename(xlm_file_path)  # -- Get base Excel path name
                        image_success_print = rf"IMG: '{img_file_name}' in C:\file_path\{img_file_path_name}"  # -- Format image success message
                        excel_success_print = rf"XLS: '{xlm_sheet_name}' in C:\file_path\{xlm_file_path_name}"  # -- Format Excel success message
                        
                        messages = [image_success_print, excel_success_print]  # -- Initialize success messages list
                        if doc_description:                             # -- Check if document description exists
                            doc_file_path_name = os.path.basename(doc_file_path)  # -- Get base document path name
                            doc_success_print = rf"DOC: '{report_name}' in C:\file_path\{doc_file_path_name}"  # -- Format document success message
                            messages.append(doc_success_print)          # -- Add document message to list

                        separator_line = '-' * max(len(msg) for msg in messages)  # -- Create separator line based on longest message

                        print(f"✔️ {report_name}")                      # -- Print report name with checkmark
                        print(separator_line)                           # -- Print separator line
                        print('\n'.join(messages))                      # -- Print success messages
                        print(separator_line)                           # -- Print separator line

                    return image_path                                   # -- Return image path

                except Exception as e:
                    print(f"⦸ Error in export_df_to_doc_image_excel: {str(e)}")  # -- Print error message
                    return None                                         # -- Return None on error
        except Exception as e:
            print(f"⦸ Error defining export_df_to_doc_image_excel: {str(e)}")  # -- Print error message for function definition
            export_df_to_doc_image_excel = lambda *args, **kwargs: None  # -- Fallback to a no-op function

        # -- Step 5: Define prepare_and_convert_df function
        def prepare_and_convert_df(DHIS2_data_key=None, hierarchy_columns=None, data_columns=None):  # -- Define function to prepare and convert DataFrame
            """
            Prepare and convert a DataFrame from DHIS2_data with available columns, applying sorting,
            default values for missing data columns, and type conversions.

            Args:
                DHIS2_data_key (str): The key to look up the DHIS2 dataset.
                hierarchy_columns (List[str] or None): List of hierarchy columns to include.
                data_columns (List[str]): List of desired data columns.

            Returns:
                Optional[pd.DataFrame]: Prepared DataFrame or None if error occurs.
            """
            try:                                                        # -- Begin try block for preparation function
                if DHIS2_data_key not in DHIS2_data:                    # -- Check if DHIS2 key exists
                    print(f"✋🏿 Error: '{DHIS2_data_key}' not found in DHIS2_data. Report not processed.")  # -- Print error message
                    return None                                         # -- Return None if key missing

                df_raw = DHIS2_data[DHIS2_data_key]                     # -- Get raw DataFrame from DHIS2 data

                if hierarchy_columns is None:                           # -- Check if hierarchy columns are provided
                    hierarchy_columns = []                              # -- Default to empty list if None

                available_columns = [col for col in data_columns if col in df_raw.columns]  # -- Filter available data columns
                if not available_columns:                               # -- Check if any requested columns are available
                    print(f"✋🏿 Warning: None of the requested columns found in '{DHIS2_data_key}'.")  # -- Print warning
                    # No return here to allow empty DataFrame with hierarchy

                df = df_raw[hierarchy_columns + available_columns].copy()  # -- Copy selected columns
                df.sort_values(by=hierarchy_columns, inplace=True, ignore_index=True)  # -- Sort by hierarchy columns

                for col in available_columns:                           # -- Iterate over available columns
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)  # -- Convert to numeric, fill NA with 0, cast to int

                for col in data_columns:                                # -- Iterate over desired columns
                    if col not in df.columns:                           # -- Check if column is missing
                        df[col] = 0                                     # -- Add missing column with default 0

                df = df[hierarchy_columns + data_columns]               # -- Reorder columns to match input order

                return df                                               # -- Return prepared DataFrame

            except Exception as e:                                      # -- Catch exceptions in preparation function
                print(f"⦸ Error preparing and converting df for '{DHIS2_data_key}': {str(e)}")  # -- Print error message
                return None                                             # -- Return None on failure
        
        # -- Step 6: Define filter_gap_and_check_empty_df function
        # -- Function: Filter gap and check empty df
        try:
            def filter_gap_and_check_empty_df(
                df=None, msg=None, opNonZero=None, opNeg=None, opPos=None, 
                opNonPos=None, opNonNeg=None, opZero=None, opLT100=None):
                """
                Filters a df based on column-specific conditions and handles empty results.
                (Docstring truncated for brevity; see original for full details.)
                """
                try:
                    if df is None or df.empty:                          # -- Check if input DataFrame is invalid
                        raise ValueError("Input df is None or empty")   # -- Raise error if None or empty
                    if not msg:                                         # -- Check if message is provided
                        raise ValueError("No message provided for empty result")  # -- Raise error if missing

                    operator_map = {                                    # -- Define operator mapping for conditions
                        'opNonZero': lambda x: x != 0,                  # -- Non-zero condition
                        'opNeg': lambda x: x < 0,                       # -- Negative condition
                        'opPos': lambda x: x > 0,                       # -- Positive condition
                        'opZero': lambda x: x == 0,                     # -- Zero condition
                        'opLT100': lambda x: x < 100                    # -- Less than 100 condition
                    }

                    conditions = []                                     # -- Initialize conditions list
                    for arg, cols in {                                  # -- Iterate over operator arguments
                        'opNonZero': opNonZero, 'opNeg': opNeg, 'opPos': opPos, 
                        'opNonPos': opNonPos, 'opNonNeg': opNonNeg, 'opZero': opZero, 'opLT100': opLT100
                    }.items():
                        if cols:                                        # -- Check if columns are provided for operator
                            for col in cols:                            # -- Iterate over columns
                                if col not in df.columns:               # -- Check if column exists
                                    raise ValueError(f"Column '{col}' not found in df")  # -- Raise error if missing
                                numeric_series = pd.to_numeric(df[col], errors='coerce').fillna(0)  # -- Convert to numeric
                                conditions.append(operator_map[arg](numeric_series))  # -- Apply operator condition

                    if not conditions:                              # -- Check if any conditions were added
                        print("No filtering conditions provided")    # -- Print warning
                        return df                                    # -- Return original DataFrame

                    combined_condition = pd.DataFrame(conditions).T.any(axis=1)  # -- Combine conditions with OR logic
                    df_filtered = df[combined_condition]            # -- Filter DataFrame

                    if df_filtered.empty:                            # -- Check if filtered DataFrame is empty
                        print("✋🏿Checked:")                        # -- Print check message
                        print("-" * len(msg))                       # -- Print separator line
                        print(msg)                                  # -- Print empty result message
                        print("-" * len(msg))                       # -- Print separator line
                        return None                                 # -- Return None for empty result

                    return df_filtered                              # -- Return filtered DataFrame

                except Exception as e:
                    print(f"⦸ Error filtering gaps and checking empty df: {str(e)}")  # -- Print error message
                    return None                                     # -- Return None on error
        except Exception as e:
            print(f"⦸ Error defining filter_gap_and_check_empty_df: {str(e)}")  # -- Print error message for function definition
            filter_gap_and_check_empty_df = lambda *args, **kwargs: None  # -- Fallback to a no-op function

         # -- Step 7: Constants Initialization for Positive Data Processing
        # -- Define column lists for HTS, AGYW, PMTCT, and KP datasets
        HTS_cols = {
            "positive": [
                "Number of people who tested HIV positive and received results (Inpatient)",  # -- Inpatient positive results
                "Number of people who tested HIV positive and received results (Outpatient)",  # -- Outpatient positive results
                "Number of people who tested HIV positive and received results (Standalone)",  # -- Standalone positive results
                "Number of people who tested HIV positive and received results (Community)"  # -- Community positive results
            ],
            "known_positive": [
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Inpatient)",  # -- Known positive inpatient
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Outpatient)",  # -- Known positive outpatient
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Standalone)",  # -- Known positive standalone
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)"  # -- Known positive community
            ]
        }

        AGYW_cols = [
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",  # -- AGYW community positive
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)"  # -- AGYW walk-in positive
        ]

        PMTCT_col = ["Number of pregnant women tested HIV positive"]  # -- PMTCT positive results

        KP_cols = [
            "HTS-3a Number of MSM that have received an HIV test during the reporting period in KP-specific programs and received HIV Positive results",  # -- MSM positive results
            "HTS-3b Number of TG that have received an HIV test during the reporting period in KP-specific programs and HIV positive results",  # -- TG positive results
            "HTS-3c Number of sex workers that have received an HIV test during the reporting period in KP-specific programs and received HIV-positive results",  # -- Sex workers positive
            "HTS-3d Number of people who inject drugs (PWID) that have received an HIV test during the reporting period in KP-specific programs and received HIV positive results",  # -- PWID positive
            "HTS-3e Number of other vulnerable populations (OVP) that have received an HIV test during the reporting period and received HIV-positive results",  # -- OVP positive
            "HTS-3f Number of people in prisons and other closed settings that have received an HIV test during the reporting period and received HIV-positive results"  # -- Prison positive
        ]

        # -- Step 8: Prepare and process HTS data
        Pre_HTS_MSF_positive = prepare_and_convert_df('HTS_MSF', MSF_hierarchy, HTS_cols["positive"] + HTS_cols["known_positive"])  # -- Prepare HTS DataFrame
        Pre_HTS_MSF_positive["HTS total tested - positive"] = Pre_HTS_MSF_positive[HTS_cols["positive"]].sum(axis=1)  # -- Calculate total positive
        Pre_HTS_MSF_positive["HTS total tested - previously known positive"] = Pre_HTS_MSF_positive[HTS_cols["known_positive"]].sum(axis=1)  # -- Calculate known positive
        Pre_HTS_MSF_positive["HTS total tested - new positive (excluding previously known)"] = np.where(
            Pre_HTS_MSF_positive["HTS total tested - previously known positive"] > 0,  # -- Check if known positive exists
            Pre_HTS_MSF_positive["HTS total tested - positive"] - Pre_HTS_MSF_positive["HTS total tested - previously known positive"],  # -- Calculate new positive
            0
        )

        # -- Step 9: Prepare AGYW data
        Pre_AGYW_MSF_positive = prepare_and_convert_df('AGYW_MSF', MSF_hierarchy, AGYW_cols)  # -- Prepare AGYW DataFrame
        Pre_AGYW_MSF_positive["AGYW total tested - new positive"] = Pre_AGYW_MSF_positive[AGYW_cols].sum(axis=1)  # -- Calculate total AGYW positive

        # -- Step 10: Prepare PMTCT data
        Pre_PMTCT_MSF_positive = prepare_and_convert_df('PMTCT_MSF', MSF_hierarchy, PMTCT_col)  # -- Prepare PMTCT DataFrame
        Pre_PMTCT_MSF_positive.rename(columns={PMTCT_col[0]: "PMTCT total tested - new positive"}, inplace=True)  # -- Rename PMTCT column

        # -- Step 11: Prepare KP data
        Pre_KP_MSF_positive = prepare_and_convert_df('KP_Prev_MSF', MSF_hierarchy, KP_cols)  # -- Prepare KP DataFrame
        Pre_KP_MSF_positive["KP_Prev total tested - new positive"] = Pre_KP_MSF_positive[KP_cols].sum(axis=1)  # -- Calculate total KP positive

        # -- Step 12: Merge all DataFrames sequentially
        base_cols = ["ReportPeriod", "Cluster", "LGA", "FacilityName"]  # -- Define base columns for merging
        Pre_MSF_positives_all = DHIS2_data["Report_Rate_Facility"][base_cols].merge(
            Pre_HTS_MSF_positive[base_cols + ["HTS total tested - new positive (excluding previously known)"]],  # -- Merge HTS data
            on=base_cols, how="left"
        ).merge(
            Pre_AGYW_MSF_positive[base_cols + ["AGYW total tested - new positive"]],  # -- Merge AGYW data
            on=base_cols, how="left"
        ).merge(
            Pre_PMTCT_MSF_positive[base_cols + ["PMTCT total tested - new positive"]],  # -- Merge PMTCT data
            on=base_cols, how="left"
        ).merge(
            Pre_KP_MSF_positive[base_cols + ["KP_Prev total tested - new positive"]],  # -- Merge KP data
            on=base_cols, how="left"
        )

        # -- Step 13: Ensure numeric types & fill NaNs
        for col in [
            "HTS total tested - new positive (excluding previously known)",  # -- HTS new positive column
            "AGYW total tested - new positive",                             # -- AGYW new positive column
            "PMTCT total tested - new positive",                            # -- PMTCT new positive column
            "KP_Prev total tested - new positive"                           # -- KP new positive column
        ]:
            Pre_MSF_positives_all[col] = pd.to_numeric(Pre_MSF_positives_all[col], errors='coerce').fillna(0).astype(int)  # -- Convert to int, fill NaNs with 0
            Pre_MSF_positives_all["Total new positive"] = Pre_MSF_positives_all[[  # -- Sum total new positive
                "HTS total tested - new positive (excluding previously known)",
                "AGYW total tested - new positive",
                "PMTCT total tested - new positive",
                "KP_Prev total tested - new positive"
            ]].sum(axis=1)

    except Exception as e:
        print(f"⦸ Error in load_functions: {str(e)}")                    # -- Print error message for load_functions

#### - Function: Get data

In [5]:
# -- Global variable to store DHIS2_data
def fetch_dhis2_data_interactive_jupyter_mode():
    """
    Interactive function to collect user inputs and fetch/process DHIS2 data in a Jupyter notebook.
    
    Args:
        None
    
    Returns:
        dict: DHIS2_data fetched from the DHIS2 server (accessible globally after submission)
    """
    global DHIS2_data  # -- Declare DHIS2_data as global to modify it within handlers
    
    # -- Step 1: Define constants
    separator_line = '-' * 43  # -- Define a static separator line of 43 dashes for formatting
    output = widgets.Output()  # -- Output area for displaying results

    # -- Step 2: Create widgets for collecting credentials
    username_input = widgets.Text(description="Username:", placeholder="Enter IHVN DHIS2 username")
    password_input = widgets.Password(description="Passkey:", placeholder="Enter IHVN DHIS2 password")
    submit_credentials = widgets.Button(description="Submit")
    credentials_box = widgets.VBox([username_input, password_input, submit_credentials])

    # -- Step 3: Create widgets for collecting periods
    start_period_input = widgets.Text(description="Start Period:", placeholder="Enter report period (YYYYMM, e.g., 202501)")
    end_period_input = widgets.Text(description="End Period:", placeholder="Enter report period (YYYYMM, e.g., 202512)")
    submit_periods = widgets.Button(description="Submit")
    periods_box = widgets.VBox([start_period_input, end_period_input, submit_periods])

    # -- Step 4: Store collected inputs
    credentials = [None, None]  # -- To store username and password
    periods = [None, None]      # -- To store start_period and end_period

    # -- Step 5: Define formatting functions
    def format_credentials(username, password):
        """Format login credentials for display with masked password."""
        username_line = f"{'Username: ':<{43 - len(username)}}{username}"
        password_line = f"{'Passkey: ':<{43 - len(password)}}{'*' * len(password)}"
        return f"{username_line}\n{password_line}"

    def format_report_period(start_period, end_period):
        """Format report period dates for display."""
        start_period_line = f"{'Period Start Date: ':<{43 - len(start_period)}}{start_period}"
        end_period_line = f"{'Period End Date: ':<{43 - len(end_period)}}{end_period}"
        return f"{start_period_line}\n{end_period_line}"

    def display_information(credentials_display, report_display):
        """Display all collected information in a formatted way."""
        with output:
            clear_output()
            print("Enter IHVN DHIS2 login credentials:")
            print(separator_line)
            print(credentials_display)
            print(separator_line)
            print()
            print('Enter report period (YYYYMM, e.g., 202501):')
            print(separator_line)
            print(report_display)
            print(separator_line)
            print()

    # -- Step 6: Define button handlers
    def on_submit_credentials(b):
        """Handle submission of credentials."""
        credentials[0] = username_input.value
        credentials[1] = password_input.value
        with output:
            clear_output()
            print("Enter IHVN DHIS2 login credentials:")
            print(separator_line)
            print(format_credentials(credentials[0], credentials[1]))
            print(separator_line)
            print()
            print('Enter report period (YYYYMM, e.g., 202501):')
            print(separator_line)
            display(periods_box)

    def on_submit_periods(b):
        """Handle submission of periods and fetch data."""
        global DHIS2_data
        periods[0] = start_period_input.value
        periods[1] = end_period_input.value

        # Display formatted info
        credentials_display = format_credentials(credentials[0], credentials[1])
        report_display = format_report_period(periods[0], periods[1])
        display_information(credentials_display, report_display)

        # -- Fetch and process DHIS2 data (placeholder)
        with output:
            DHIS2_data = fetch_and_process_DHIS2_data(credentials[0], credentials[1], periods[0], periods[1])
            load_variables()
            load_functions()

            # -- Optional LGA Filter Widget Setup
            if "Report_Rate_LGA" in DHIS2_data and "LGA" in DHIS2_data["Report_Rate_LGA"].columns:
                available_lgas = sorted(DHIS2_data["Report_Rate_LGA"]["LGA"].dropna().unique())

                lga_filter_widget = widgets.SelectMultiple(
                    options=available_lgas,
                    description="Select LGA:",
                    #layout=widgets.Layout(width='300px', height='150px'),
                    rows=6
                )

                apply_filter_button = widgets.Button(description="Apply report level")

                def on_apply_filter_clicked(b):
                    selected_lgas = list(lga_filter_widget.value)
                    with output:
                        if not selected_lgas:
                            print("✔️ State level data ready")
                        else:
                            for key, df in DHIS2_data.items():
                                if isinstance(df, pd.DataFrame) and "LGA" in df.columns:
                                    DHIS2_data[key] = df[df["LGA"].isin(selected_lgas)].copy()
                            print(f"✔️ LGA level data ready for {selected_lgas}")

                apply_filter_button.on_click(on_apply_filter_clicked)

                print(f"\nOptional: Select report level - LGA")
                print(separator_line)
                display(widgets.VBox([
                    lga_filter_widget,
                    apply_filter_button
                ]))
                print(separator_line)
                
    # -- Step 7: Link buttons to handlers
    submit_credentials.on_click(on_submit_credentials)
    submit_periods.on_click(on_submit_periods)

    # -- Step 8: Display the initial interface
    with output:
        print("Enter IHVN DHIS2 login credentials:")
        print(separator_line)
        display(credentials_box)
    display(output)

### MSF reporting rate

#### - Reporting rate: LGA

In [6]:
# -- Define the main function to process LGA report rate gap
def process_lga_report_rate_gap(display_output=None):
    """
    Process LGA report rate gap, exporting results as image and Excel files.
    Caches the styled df and df shape for faster display in subsequent calls.
    Reprocesses if the df shape changes.
    
    Args:
        display_output (bool, optional): If True, displays the styled df for LGAs with gap.
            Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        MSF_report_rate_columns = [                           # -- Define list of MSF report rate columns
            'AGYW Monthly Summary Form - Reporting rate',
            'ART MSF - Reporting rate',
            'Care & Support MSF - Reporting rate',
            'HTS Summary Form - Reporting rate',
            'NSP Summary Form - Reporting rate',
            'PMTCT MSF - Reporting rate',
            'Prevention Summary Form - Reporting rate'
        ]
        MSF_report_rate_msg = "No ANSO report rate gap found"  # -- Define message for no gaps
        report_name = "ANSO MSF report rate"                  # -- Define report name

        # -- Step 2: Prepare data
        df_Report_Rate_LGA = prepare_and_convert_df(          # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='Report_Rate_LGA',                 # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=MSF_report_rate_columns              # -- Include specified report rate columns
        )
        if df_Report_Rate_LGA is None:                        # -- Check if data preparation failed or DataFrame is empty
            return                                            # -- Exit function if no data

        # -- Step 3: Set export variables
        report_month = df_Report_Rate_LGA['ReportPeriod'].iloc[0]  # -- Extract report month from DataFrame
        report_sheet_name = "All LGAs"                        # -- Define Excel sheet name
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name

        # -- Step 4: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_lga_report_rate_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_Report_Rate_LGA.shape      # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print("-" * len(cached_display_name))     # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print("-" * len(cached_display_name))     # -- Print separator line
                    display(process_lga_report_rate_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 5: Filter for gaps
        df_Report_Rate_LGA_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_Report_Rate_LGA,                            # -- Input DataFrame
            msg=MSF_report_rate_msg,                          # -- Message for empty result
            opNonZero=None,                                   # -- No non-zero filter
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=MSF_report_rate_columns                   # -- Filter for values less than 100
        )
        if df_Report_Rate_LGA_gap is None:                    # -- Check if no gaps found
            if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cache exists
                del process_lga_report_rate_gap.cached_style  # -- Clear cached style
            if hasattr(process_lga_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_lga_report_rate_gap.cached_shape  # -- Clear cached shape
            return                                            # -- Exit function

        # -- Step 6: Style the DataFrame
        df_Report_Rate_LGA_style = (                          # -- Apply styling to filtered DataFrame
            df_Report_Rate_LGA_gap.style                      # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=MSF_report_rate_columns)  # -- Highlight outliers in red for report rate columns
            .map(outlier_green)                               # -- Apply green outlier styling (assumed general application)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_lga_report_rate_gap.cached_style = df_Report_Rate_LGA_style  # -- Cache styled DataFrame
        process_lga_report_rate_gap.cached_shape = df_Report_Rate_LGA.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to image and Excel formats
            report_name=report_name,                          # -- Pass report name
            df_shape=df_Report_Rate_LGA_gap,                  # -- Pass filtered DataFrame for shape
            df_style=df_Report_Rate_LGA_style,                # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=sub_folder2_image_file_report_rate, # -- Pass image file path
            doc_description=None,                             # -- No document description (not used)
            doc_indicators_to_italicize=None,                 # -- No indicators to italicize (not used)
            doc_indicators_to_underline=None,                 # -- No indicators to underline (not used)
            xlm_file_path=doc_file_report_rate_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 9: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            display(df_Report_Rate_LGA_style)                 # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing LGA report rate gap: {str(e)}")  # -- Print error message
        if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cache exists
            del process_lga_report_rate_gap.cached_style      # -- Clear cached style
        if hasattr(process_lga_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_lga_report_rate_gap.cached_shape      # -- Clear cached shape

#### - Reporting rate: Facility

In [7]:
# -- Define the main function to process facility report rate gap
def process_facility_report_rate_gap(display_output=None):
    """
    Process facility report rate gaps for each LGA, exporting results as images and Excel files.
    Caches styled dfs for each LGA and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the styled df images for each LGA with gaps.
            Defaults to None (will treat as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        MSF_report_rate_columns = [                           # -- Define list of MSF report rate columns
            'AGYW Monthly Summary Form - Reporting rate',
            'ART MSF - Reporting rate',
            'Care & Support MSF - Reporting rate',
            'HTS Summary Form - Reporting rate',
            'NSP Summary Form - Reporting rate',
            'PMTCT MSF - Reporting rate',
            'Prevention Summary Form - Reporting rate'
        ]

        # -- Step 2: Prepare data
        df_Report_Rate_Facility = prepare_and_convert_df(     # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='Report_Rate_Facility',            # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=MSF_report_rate_columns              # -- Include specified report rate columns
        )
        if df_Report_Rate_Facility is None:                   # -- Check if data preparation failed or DataFrame is empty
            return                                            # -- Exit function if no data

        # -- Step 3: Check and display cached styled DataFrames
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_facility_report_rate_gap, 'cached_styles'):  # -- Check if cached styled DataFrames exist
                cached_shape = getattr(process_facility_report_rate_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_Report_Rate_Facility.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    for lga, style in process_facility_report_rate_gap.cached_styles.items():  # -- Iterate over cached styles
                        cached_display_name = f"✔️ Displaying {lga} facility report rate gap "  # -- Define display message for LGA
                        print("-" * len(cached_display_name)) # -- Print separator line
                        print(cached_display_name)            # -- Print display message
                        print("-" * len(cached_display_name)) # -- Print separator line
                        display(style)                        # -- Display cached styled DataFrame for LGA
                    return                                    # -- Exit function

        # -- Step 4: Initialize cache
        if not hasattr(process_facility_report_rate_gap, 'cached_styles'):  # -- Check if cache attribute exists
            process_facility_report_rate_gap.cached_styles = {}  # -- Initialize dictionary to store styled DataFrames per LGA

        # -- Step 5: Identify unique LGAs
        lga_list = pd.Series(df_Report_Rate_Facility['LGA'].unique())  # -- Extract unique LGA names as a Series

        # -- Step 6: Process each LGA for report rate gaps
        for current_lga in lga_list:                          # -- Iterate over each unique LGA
            # -- Step 6.1: Filter DataFrame for current LGA
            lga_filtered = df_Report_Rate_Facility[df_Report_Rate_Facility['LGA'] == current_lga]  # -- Filter DataFrame to current LGA

            MSF_report_rate_msg = f"No {current_lga} report rate gap found"  # -- Define message for no gaps

            # -- Step 6.2: Apply filtering for gaps
            lga_filtered_gap = filter_gap_and_check_empty_df(  # -- Filter LGA-specific subset for gaps
                df=lga_filtered,                              # -- Input LGA-filtered DataFrame
                msg=MSF_report_rate_msg,                      # -- Message for empty result
                opNonZero=None,                               # -- No non-zero filter
                opNeg=None,                                   # -- No negative filter
                opPos=None,                                   # -- No positive filter
                opZero=None,                                  # -- No zero filter
                opLT100=MSF_report_rate_columns               # -- Filter for values less than 100
            )

            if lga_filtered_gap is None:                      # -- Check if no gaps found for this LGA
                if current_lga in process_facility_report_rate_gap.cached_styles:  # -- Check if LGA is in cache
                    del process_facility_report_rate_gap.cached_styles[current_lga]  # -- Remove LGA from cache
                continue                                      # -- Skip to next LGA

            # -- Step 6.3: Style the DataFrame
            lga_filtered_style = (                            # -- Apply styling to filtered LGA DataFrame
                lga_filtered_gap.style                        # -- Start with DataFrame style object
                .hide(axis='index')                           # -- Hide index column
                .map(outlier_red, subset=MSF_report_rate_columns)  # -- Highlight outliers in red for report rate columns
                .map(outlier_green)                           # -- Apply green outlier styling (assumed general application)
            )

            # -- Step 6.4: Cache the styled DataFrame for this LGA
            process_facility_report_rate_gap.cached_styles[current_lga] = lga_filtered_style  # -- Store styled DataFrame in cache

            # -- Step 6.5: Define export variables
            report_name = f"{current_lga} Facility Report Rate Gap"  # -- Define report name for current LGA
            report_image_name = f"{current_lga}.png"          # -- Define image file name for current LGA
            report_sheet_name = f"{current_lga}"              # -- Define Excel sheet name for current LGA

            # -- Step 6.6: Export results
            export_df_to_doc_image_excel(                     # -- Export LGA-specific DataFrame to image and Excel
                report_name=report_name,                      # -- Pass report name
                df_shape=lga_filtered_gap,                    # -- Pass filtered LGA DataFrame for shape
                df_style=lga_filtered_style,                  # -- Pass styled LGA DataFrame
                img_file_name=report_image_name,              # -- Pass image file name
                img_file_path=sub_folder2_image_file_report_rate,  # -- Pass image file path
                doc_description=None,                         # -- No document description (not used)
                doc_indicators_to_italicize=None,             # -- No indicators to italicize (not used)
                doc_indicators_to_underline=None,             # -- No indicators to underline (not used)
                xlm_file_path=doc_file_report_rate_xlsx,      # -- Pass Excel file path
                xlm_sheet_name=report_sheet_name              # -- Pass Excel sheet name
            )

            # -- Step 6.7: Optionally display styled DataFrame
            if display_output:                                # -- Check if display is requested
                display(lga_filtered_style)                   # -- Display styled DataFrame for current LGA

        # -- Step 7: Cache overall unfiltered DataFrame shape
        process_facility_report_rate_gap.cached_shape = df_Report_Rate_Facility.shape  # -- Cache unfiltered DataFrame shape after processing

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing facility report rate gaps: {str(e)}")  # -- Print error message
        if hasattr(process_facility_report_rate_gap, 'cached_styles'):  # -- Check if cache exists
            process_facility_report_rate_gap.cached_styles.clear()  # -- Clear cached styles dictionary
        if hasattr(process_facility_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_facility_report_rate_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function on error

### AGYW

#### - AGYW HTS

In [8]:
# -- Define the main function to process AGYW HTS gap
def process_AGYW_HTS_gap(display_output=None):
    """
    Process AGYW HTS gap for each LGA, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for LGAs with gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_HTS_columns = [                                  # -- Define list of AGYW HTS columns in desired order
            "Number of AGYW reached with HIV Prevention Program - defined package of service during the reporting period (Community)",
            "Number of AGYW reached with HIV Prevention Program - defined package of service during the reporting (Walk-In)",
            "Number of AGYW that received an HIV test during the reporting period and know their status (Community)",
            "Number of AGYW that received an HIV test during the reporting period and know their status (Walk-In)"
        ]
        name = "AGYW HTS gap"                                 # -- Define general name
        AGYW_HTS_gap_columns = ['AGYW HTS gap']               # -- Define gap column name
        AGYW_HTS_msg = f"No {name}"                           # -- Define message for no gaps
        report_name = f"{name}1"                              # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_HTS = prepare_and_convert_df(                 # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW_MSF',                        # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=AGYW_HTS_columns                     # -- Include specified AGYW HTS columns
        )
        if df_AGYW_HTS is None:                               # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        # -- Step 2.1: Add missing columns with default value and maintain order
        for col in AGYW_HTS_columns:                          # -- Iterate over required columns
            if col not in df_AGYW_HTS.columns:                # -- Check if column is missing
                df_AGYW_HTS[col] = 0                          # -- Add missing column with default value 0

        # -- Step 2.2: Reorder columns to match AGYW_HTS_columns plus hierarchy
        all_columns = MSF_hierarchy + AGYW_HTS_columns         # -- Combine MSF_hierarchy and AGYW HTS columns in desired order
        df_AGYW_HTS = df_AGYW_HTS.reindex(columns=all_columns)  # -- Reorder DataFrame columns to match all_columns

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_AGYW_HTS_gap, 'cached_style'): # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_HTS_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_HTS.shape             # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    display(process_AGYW_HTS_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW reached with HIV Prevention
        df_AGYW_HTS["Total AGYW reached with HIV Prevention"] = (  # -- Calculate total reached
            df_AGYW_HTS[AGYW_HTS_columns[0]] + df_AGYW_HTS[AGYW_HTS_columns[1]]  # -- Sum Community and Walk-In
        )
        # -- Step 4.2: Total AGYW received HIV test & know status
        df_AGYW_HTS["Total AGYW received HIV test & know status"] = (  # -- Calculate total tested
            df_AGYW_HTS[AGYW_HTS_columns[2]] + df_AGYW_HTS[AGYW_HTS_columns[3]]  # -- Sum Community and Walk-In
        )
        # -- Step 4.3: AGYW HTS gap
        df_AGYW_HTS[AGYW_HTS_gap_columns[0]] = np.where(      # -- Calculate gap
            df_AGYW_HTS["Total AGYW received HIV test & know status"] > df_AGYW_HTS["Total AGYW reached with HIV Prevention"],  # -- Condition for gap
            df_AGYW_HTS["Total AGYW received HIV test & know status"] - df_AGYW_HTS["Total AGYW reached with HIV Prevention"],  # -- Positive gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 5: Filter and validate gaps
        df_AGYW_HTS_gap = filter_gap_and_check_empty_df(      # -- Filter DataFrame for gaps
            df=df_AGYW_HTS,                                   # -- Input DataFrame
            msg=AGYW_HTS_msg,                                 # -- Message for empty result
            opNonZero=AGYW_HTS_gap_columns,                   # -- Filter for non-zero gaps
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=None                                      # -- No less-than-100 filter
        )
        if df_AGYW_HTS_gap is None:                           # -- Check if no gaps found
            if hasattr(process_AGYW_HTS_gap, 'cached_style'): # -- Check if cache exists
                del process_AGYW_HTS_gap.cached_style         # -- Clear cached style
            if hasattr(process_AGYW_HTS_gap, 'cached_shape'): # -- Check if cached shape exists
                del process_AGYW_HTS_gap.cached_shape         # -- Clear cached shape
            return                                            # -- Exit function

        # -- Step 6: Style the DataFrame
        df_AGYW_HTS_gap_style = (                             # -- Apply styling to filtered DataFrame
            df_AGYW_HTS_gap.style                             # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=AGYW_HTS_gap_columns)    # -- Highlight outliers in gap column
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_HTS_gap.cached_style = df_AGYW_HTS_gap_style  # -- Cache styled DataFrame
        process_AGYW_HTS_gap.cached_shape = df_AGYW_HTS.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_HTS['ReportPeriod'].iloc[0]    # -- Extract report month from DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # -- Define image file path
        report_sheet_name = report_name                       # -- Define Excel sheet name

        # -- Step 9: Create description for Word document
        if (df_AGYW_HTS[AGYW_HTS_gap_columns[0]] != 0).any():  # -- Check if any gaps exist
            report_description = (                                # -- Define report description
                f"Report Name: {AGYW_HTS_gap_columns[0]}"
                f"\n{AGYW_HTS_columns[2]}\nplus {AGYW_HTS_columns[3]}"
                f"\nshould not be greater than"
                f"\n{AGYW_HTS_columns[0]}\nplus {AGYW_HTS_columns[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to multiple formats
            report_name=report_name,                          # -- Pass report name
            df_shape=df_AGYW_HTS,                             # -- Pass unfiltered DataFrame for shape
            df_style=df_AGYW_HTS_gap_style,                   # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=report_image_path,                  # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,          # -- Pass Word document path
            doc_description=report_description,               # -- Pass description
            doc_indicators_to_italicize=AGYW_HTS_columns,     # -- Italicize AGYW HTS columns
            doc_indicators_to_underline=AGYW_HTS_gap_columns, # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            display(df_AGYW_HTS_gap_style)                    # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_HTS_gap, 'cached_style'):     # -- Check if cache exists
            del process_AGYW_HTS_gap.cached_style             # -- Clear cached style
        if hasattr(process_AGYW_HTS_gap, 'cached_shape'):     # -- Check if cached shape exists
            del process_AGYW_HTS_gap.cached_shape             # -- Clear cached shape
        return                                                # -- Exit function

#### - AGYW Positive

In [9]:
# -- Define the main function to process AGYW Positive gap
def process_AGYW_Positive_gap(display_output=None):
    """
    Process AGYW Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_Positive_columns = [                             # -- Define list of AGYW Positive columns in desired order
            "Number of AGYW that received an HIV test during the reporting period and know their status (Community)",
            "Number of AGYW that received an HIV test during the reporting period and know their status (Walk-In)",
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)"
        ]
        name = "AGYW Positive gap"                            # -- Define general name
        AGYW_Positive_gap_columns = ["AGYW tested Positive gap"]  # -- Define gap column name
        AGYW_Positive_msg = f"No {name}"                      # -- Define message for no gaps
        report_name = f"{name}2"                              # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_Positive = prepare_and_convert_df(            # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW_MSF',                        # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=AGYW_Positive_columns                # -- Include specified AGYW Positive columns
        )
        if df_AGYW_Positive is None:                          # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        # -- Step 2.1: Add missing columns with default value
        for col in AGYW_Positive_columns:                     # -- Iterate over required columns
            if col not in df_AGYW_Positive.columns:           # -- Check if column is missing
                df_AGYW_Positive[col] = 0                     # -- Add missing column with default value 0

        # -- Step 2.2: Reorder columns to match AGYW_Positive_columns plus hierarchy
        all_columns = MSF_hierarchy + AGYW_Positive_columns   # -- Combine MSF_hierarchy and AGYW Positive columns in desired order
        df_AGYW_Positive = df_AGYW_Positive.reindex(columns=all_columns)  # -- Reorder DataFrame columns to match all_columns

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_Positive_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_Positive.shape        # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    display(process_AGYW_Positive_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW tested
        df_AGYW_Positive["Total AGYW tested"] = (             # -- Calculate total tested
            df_AGYW_Positive[AGYW_Positive_columns[0]] +      # -- Add Community tested
            df_AGYW_Positive[AGYW_Positive_columns[1]]        # -- Add Walk-In tested
        )
        # -- Step 4.2: Total AGYW tested positive
        df_AGYW_Positive["Total AGYW tested positive"] = (    # -- Calculate total tested positive
            df_AGYW_Positive[AGYW_Positive_columns[2]] +      # -- Add Community positive
            df_AGYW_Positive[AGYW_Positive_columns[3]]        # -- Add Walk-In positive
        )
        # -- Step 4.3: AGYW tested Positive gap
        df_AGYW_Positive[AGYW_Positive_gap_columns[0]] = np.where(  # -- Calculate gap
            df_AGYW_Positive["Total AGYW tested positive"] > df_AGYW_Positive["Total AGYW tested"],  # -- Condition for gap
            df_AGYW_Positive["Total AGYW tested positive"] - df_AGYW_Positive["Total AGYW tested"],  # -- Positive gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 5: Filter and validate gaps
        df_AGYW_Positive_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_AGYW_Positive,                              # -- Input DataFrame
            msg=AGYW_Positive_msg,                            # -- Message for empty result
            opNonZero=AGYW_Positive_gap_columns,              # -- Filter for non-zero gaps
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=None                                      # -- No less-than-100 filter
        )
        if df_AGYW_Positive_gap is None:                      # -- Check if no gaps found
            if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # -- Check if cache exists
                del process_AGYW_Positive_gap.cached_style    # -- Clear cached style
            if hasattr(process_AGYW_Positive_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_AGYW_Positive_gap.cached_shape    # -- Clear cached shape
            return                                            # -- Exit function

        # -- Step 6: Style the DataFrame
        df_AGYW_Positive_gap_style = (                        # -- Apply styling to filtered DataFrame
            df_AGYW_Positive_gap.style                        # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=AGYW_Positive_gap_columns)  # -- Highlight outliers in gap column
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_Positive_gap.cached_style = df_AGYW_Positive_gap_style  # -- Cache styled DataFrame
        process_AGYW_Positive_gap.cached_shape = df_AGYW_Positive.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_Positive_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # -- Define image file path
        report_sheet_name = report_name                       # -- Define Excel sheet name

        # -- Step 9: Create description for Word document
        if (df_AGYW_Positive_gap[AGYW_Positive_gap_columns[0]] != 0).any():  # -- Check if any gaps exist
            report_description = (                            # -- Define report description if gaps present
                f"Report Name: {AGYW_Positive_gap_columns[0]}"
                f"\n{AGYW_Positive_columns[2]}\nplus {AGYW_Positive_columns[3]}"
                f"\nshould not be greater than"
                f"\n{AGYW_Positive_columns[0]}\nplus {AGYW_Positive_columns[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to multiple formats
            report_name=report_name,                          # -- Pass report name
            df_shape=df_AGYW_Positive_gap,                    # -- Pass filtered DataFrame for shape
            df_style=df_AGYW_Positive_gap_style,              # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=report_image_path,                  # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,          # -- Pass Word document path
            doc_description=report_description,               # -- Pass description
            doc_indicators_to_italicize=AGYW_Positive_columns,  # -- Italicize AGYW Positive columns
            doc_indicators_to_underline=AGYW_Positive_gap_columns,  # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            display(df_AGYW_Positive_gap_style)               # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # -- Check if cache exists
            del process_AGYW_Positive_gap.cached_style        # -- Clear cached style
        if hasattr(process_AGYW_Positive_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_AGYW_Positive_gap.cached_shape        # -- Clear cached shape
        return                                            # -- Exit function

#### - AGYW Linkage

In [10]:
# -- Define the main function to process AGYW Positive Linkage gap
def process_AGYW_Positive_Linkage_gap(display_output=None):
    """
    Process AGYW Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_Positive_Linkage_columns = [                     # -- Define list of AGYW Positive Linkage columns in desired order
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)",
            "Total number of AGYW who tested HIV Positive and are successfully linked to treatment during the reporting period (Community & Walk-In)",
            "Linked/Referred for treatment to GF supported site (subset of 4)",
            "Linked/Referred for treatment to non-GF supported site (subset of 4)",
            "Number of AGYW newly started on ART during the reporting period"
        ] 
        name = "AGYW Positive Linkage gap"                    # -- Define general name
        AGYW_Positive_Linkage_gap_columns = [                 # -- Define list of gap column names
            "AGYW positive linked to treatment gap",
            "AGYW positive linkage to GF/non-GF supported site gap",
            "AGYW newly started on ART gap"
        ]
        AGYW_Positive_Linkage_msg = f"No {name}"              # -- Define message for no gaps
        report_name = f"{name}3"                              # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_Positive_Linkage = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW_MSF',                        # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=AGYW_Positive_Linkage_columns        # -- Include specified AGYW Positive Linkage columns
        )
        if df_AGYW_Positive_Linkage is None:                  # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        # -- Step 2.1: Add missing columns with default value
        for col in AGYW_Positive_Linkage_columns:             # -- Iterate over required columns
            if col not in df_AGYW_Positive_Linkage.columns:   # -- Check if column is missing
                df_AGYW_Positive_Linkage[col] = 0             # -- Add missing column with default value 0

        # -- Step 2.2: Reorder columns to match AGYW_Positive_Linkage_columns plus hierarchy
        all_columns = MSF_hierarchy + AGYW_Positive_Linkage_columns  # -- Combine hierarchy and AGYW Positive Linkage columns in desired order
        df_AGYW_Positive_Linkage = df_AGYW_Positive_Linkage.reindex(columns=all_columns)  # -- Reorder DataFrame columns to match all_columns

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_Positive_Linkage_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_Positive_Linkage.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    display(process_AGYW_Positive_Linkage_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW Tested Positive
        df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] = (  # -- Calculate total tested positive
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[0]] +  # -- Add Community positive
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[1]]    # -- Add Walk-In positive
        )
        # -- Step 4.2: AGYW positive linked to treatment gap
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[0]] = np.where(  # -- Calculate linkage gap
            df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[2]],  # -- Condition for gap
            df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[2]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )
        # -- Step 4.3: AGYW positive linkage to GF/non-GF supported site gap
        total_linked_to_sites = (                             # -- Precompute total linked to sites
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[3]] +  # -- Add GF supported site
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[4]]    # -- Add non-GF supported site
        )
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[1]] = np.where(  # -- Calculate site linkage gap
            total_linked_to_sites != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[2]],  # -- Condition for gap
            total_linked_to_sites - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[2]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )
        # -- Step 4.4: AGYW newly started on ART gap
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[2]] = np.where(  # -- Calculate ART gap
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[2]] != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[5]],  # -- Condition for gap
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[2]] - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns[5]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 5: Filter and validate gaps
        df_AGYW_Positive_Linkage_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_AGYW_Positive_Linkage,                      # -- Input DataFrame
            msg=AGYW_Positive_Linkage_msg,                    # -- Message for empty result
            opNonZero=AGYW_Positive_Linkage_gap_columns,      # -- Filter for non-zero gaps
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=None                                      # -- No less-than-100 filter
        )
        if df_AGYW_Positive_Linkage_gap is None:              # -- Check if no gaps found
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # -- Check if cache exists
                del process_AGYW_Positive_Linkage_gap.cached_style  # -- Clear cached style
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_AGYW_Positive_Linkage_gap.cached_shape  # -- Clear cached shape
            return                                            # -- Exit function

        # -- Step 6: Style the DataFrame
        df_AGYW_Positive_Linkage_gap_style = (                # -- Apply styling to filtered DataFrame
            df_AGYW_Positive_Linkage_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=AGYW_Positive_Linkage_gap_columns)  # -- Highlight outliers in gap columns
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_Positive_Linkage_gap.cached_style = df_AGYW_Positive_Linkage_gap_style  # -- Cache styled DataFrame
        process_AGYW_Positive_Linkage_gap.cached_shape = df_AGYW_Positive_Linkage.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_Positive_Linkage_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # -- Define image file path
        report_sheet_name = report_name                       # -- Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        report_description = []                               # -- Initialize list to collect descriptions
        # -- Step 9.1: Add description for AGYW Positive Linkage gap
        if (df_AGYW_Positive_Linkage_gap[AGYW_Positive_Linkage_gap_columns[0]] != 0).any():  # -- Check if linkage gap exists
            report_description += (                           # -- Add description for linkage gap
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[0]}"
                f"\n{AGYW_Positive_Linkage_columns[0]}\nplus {AGYW_Positive_Linkage_columns[1]}"
                f"\nshould be equal to {AGYW_Positive_Linkage_columns[2]}"
                f"\nNote: Where this AGYW linkage gap is true, please ignore the outlier."
            )
        # -- Step 9.2: Add description for AGYW Positive Linkage to GF/non-GF supported site gap
        if (df_AGYW_Positive_Linkage_gap[AGYW_Positive_Linkage_gap_columns[1]] != 0).any():  # -- Check if site linkage gap exists
            report_description.append(                        # -- Add description for site linkage gap
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[1]}"
                f"\n{AGYW_Positive_Linkage_columns[3]}\nplus {AGYW_Positive_Linkage_columns[4]}"
                f"\nshould be equal to {AGYW_Positive_Linkage_columns[2]}"
            )
        # -- Step 9.3: Add description for AGYW newly started on ART gap
        if (df_AGYW_Positive_Linkage_gap[AGYW_Positive_Linkage_gap_columns[2]] != 0).any():  # -- Check if ART gap exists
            report_description.append(                        # -- Add description for ART gap
                f"Report Name: {AGYW_Positive_linkage_gap_columns[2]}"
                f"\n{AGYW_Positive_Linkage_columns[5]}"
                f"\nshould be equal to {AGYW_Positive_Linkage_columns[2]}"
            )
        # -- Step 9.4: Join all descriptions
        report_description = "\n".join(report_description)

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to multiple formats
            report_name=report_name,                          # -- Pass report name
            df_shape=df_AGYW_Positive_Linkage_gap,            # -- Pass filtered DataFrame for shape
            df_style=df_AGYW_Positive_Linkage_gap_style,      # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=report_image_path,                  # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,          # -- Pass Word document path
            doc_description=report_description,               # -- Pass description
            doc_indicators_to_italicize=AGYW_Positive_Linkage_columns,  # -- Italicize AGYW Positive Linkage columns
            doc_indicators_to_underline=AGYW_Positive_Linkage_gap_columns,  # -- Underline gap columns
            xlm_file_path=doc_file_msf_outlier_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            display(df_AGYW_Positive_Linkage_gap_style)       # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # -- Check if cache exists
            del process_AGYW_Positive_Linkage_gap.cached_style  # -- Clear cached style
        if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_AGYW_Positive_Linkage_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function

#### - AGYW TB Screening

In [11]:
# -- Define the main function to process AGYW TB Screening gap
def process_AGYW_TB_Screening_gap(display_output=None):
    """
    Process AGYW TB Screening gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_TB_Screening_columns = [                     # -- Define list of AGYW TB Screening columns in desired order
            "Number of AGYW newly started on ART during the reporting period",
            "Number of AGYW screened for TB amongst those newly started on ART during the reporting period"
        ]
        name = "AGYW TB Screening gap"                    # -- Define general name
        AGYW_TB_Screening_gap_columns = ["AGYW TB screening gap"]  # -- Define list of gap column names
        AGYW_TB_Screening_msg = f"No {name}"              # -- Define message for no gaps
        report_name = f"{name}4"                          # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_TB_Screening = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW_MSF',                    # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,              # -- Use MSF hierarchy columns
            data_columns=AGYW_TB_Screening_columns        # -- Include specified AGYW TB Screening columns
        )
        if df_AGYW_TB_Screening is None:                  # -- Check if data preparation failed
            return                                        # -- Exit function if no data

        # -- Step 2.1: Add missing columns with default value
        for col in AGYW_TB_Screening_columns:             # -- Iterate over required columns
            if col not in df_AGYW_TB_Screening.columns:   # -- Check if column is missing
                df_AGYW_TB_Screening[col] = 0             # -- Add missing column with default value 0

        # -- Step 2.2: Reorder columns to match AGYW_TB_Screening_columns plus hierarchy
        all_columns = MSF_hierarchy + AGYW_TB_Screening_columns  # -- Combine MSF_hierarchy and AGYW TB Screening columns in desired order
        df_AGYW_TB_Screening = df_AGYW_TB_Screening.reindex(columns=all_columns)  # -- Reorder DataFrame columns to match all_columns

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display is requested
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_TB_Screening_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_TB_Screening.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:           # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))  # -- Print separator line
                    print(cached_display_name)              # -- Print display message
                    print(f"-" * len(cached_display_name))  # -- Print separator line
                    display(process_AGYW_TB_Screening_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                  # -- Exit function

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: AGYW TB Screening gap
        df_AGYW_TB_Screening[AGYW_TB_Screening_gap_columns[0]] = np.where(  # -- Calculate TB Screening gap
            df_AGYW_TB_Screening[AGYW_TB_Screening_columns[1]] != df_AGYW_TB_Screening[AGYW_TB_Screening_columns[0]],  # -- Condition for gap
            df_AGYW_TB_Screening[AGYW_TB_Screening_columns[1]] - df_AGYW_TB_Screening[AGYW_TB_Screening_columns[0]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 5: Filter and validate gaps
        df_AGYW_TB_Screening_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_AGYW_TB_Screening,                      # -- Input DataFrame
            msg=AGYW_TB_Screening_msg,                    # -- Message for empty result
            opNonZero=AGYW_TB_Screening_gap_columns,      # -- Filter for non-zero gaps
            opNeg=None,                                   # -- No negative filter
            opPos=None,                                   # -- No positive filter
            opZero=None,                                  # -- No zero filter
            opLT100=None                                  # -- No less-than-100 filter
        )
        if df_AGYW_TB_Screening_gap is None:              # -- Check if no gaps found
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
                del process_AGYW_TB_Screening_gap.cached_style  # -- Clear cached style
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_AGYW_TB_Screening_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function

        # -- Step 6: Style the DataFrame
        df_AGYW_TB_Screening_gap_style = (                # -- Apply styling to filtered DataFrame
            df_AGYW_TB_Screening_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                           # -- Hide index column
            .map(outlier_red, subset=AGYW_TB_Screening_gap_columns)  # -- Highlight outliers in gap columns
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_TB_Screening_gap.cached_style = df_AGYW_TB_Screening_gap_style  # -- Cache styled DataFrame
        process_AGYW_TB_Screening_gap.cached_shape = df_AGYW_TB_Screening.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_TB_Screening_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # -- Define image file path
        report_sheet_name = report_name                   # -- Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        # -- Step 9.1: Add description for AGYW Positive Linkage gap
        if (df_AGYW_TB_Screening_gap[AGYW_TB_Screening_gap_columns[0]] != 0).any():  # -- Check if TB Screening gap exists
            report_description = (                        # -- Add description for TB Screening gap
                f"Report Name: {AGYW_TB_Screening_gap_columns[0]}"
                f"\n{AGYW_TB_Screening_columns[1]}\nshould be equal to {AGYW_TB_Screening_columns[0]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                     # -- Export DataFrame to multiple formats
            report_name=report_name,                      # -- Pass report name
            df_shape=df_AGYW_TB_Screening_gap,            # -- Pass filtered DataFrame for shape
            df_style=df_AGYW_TB_Screening_gap_style,      # -- Pass styled DataFrame
            img_file_name=report_image_name,              # -- Pass image file name
            img_file_path=report_image_path,              # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,      # -- Pass Word document path
            doc_description=report_description,           # -- Pass description
            doc_indicators_to_italicize=AGYW_TB_Screening_columns,  # -- Italicize AGYW Positive Linkage columns
            doc_indicators_to_underline=AGYW_TB_Screening_gap_columns,  # -- Underline gap columns
            xlm_file_path=doc_file_msf_outlier_xlsx,      # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name              # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                # -- Check if display is requested
            display(df_AGYW_TB_Screening_gap_style)       # -- Display styled DataFrame

    except Exception as e:                                # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
            del process_AGYW_TB_Screening_gap.cached_style  # -- Clear cached style
        if hasattr(process_AGYW_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_AGYW_TB_Screening_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function

### ART

#### - ART Linkage

In [12]:
# -- Define the main function to process ART positive and enrollment gap
def process_ART_PosEnrolment_gap(display_output=None):          # -- Define function
    """
    Process ART Positive and Enrolment gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_PosEnrolment_columns = [                       # -- Define list of ART positive and enrollment columns
            "ART 1: Number of HIV positive persons newly enrolled in clinical care during the month",
            "ART 2: Number of people living with HIV newly started on ART during the month (excludes ART transfer-in)"
        ]
        ART_PosEnrolment_columns_desc = ART_PosEnrolment_columns + ["Total Tested Positive"]  # -- Extended list for description
        name = "ART Positive-Enrolment gap"                # -- General report name
        ART_PosEnrolment_gap_columns = ["ART Enrolment gap", "ART Linkage gap"]  # -- Gap column names
        report_name = f"{name}5"                           # -- Report name with suffix

        # -- Step 2: Prepare data
        df_ART_PosEnrolment = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key='ART_MSF',                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_PosEnrolment_columns          # -- ART columns
        )

        # -- Step 3: Merge with external DataFrame
        df_ART_PosEnrolment = Pre_MSF_positives_all.merge(  # -- Merge with positives data
            df_ART_PosEnrolment,                           # -- Merge target
            on=["ReportPeriod", "Cluster", "LGA", "FacilityName"],  # -- Merge keys
            how="right"                                    # -- Keep all rows from df_ART_PosEnrolment
        )
        df_ART_PosEnrolment = df_ART_PosEnrolment.fillna(0)  # -- Step 3.1: Fill NaN with 0
        float_columns = df_ART_PosEnrolment.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                          # -- Step 3.2: Cast float to int
            df_ART_PosEnrolment[col] = df_ART_PosEnrolment[col].astype(int)

        # -- Step 6: Calculate derived metrics
        df_ART_PosEnrolment[ART_PosEnrolment_gap_columns[0]] = np.where(  # -- Step 6.1: ART enrolment gap
            df_ART_PosEnrolment[ART_PosEnrolment_columns[0]] != df_ART_PosEnrolment["Total new positive"],
            df_ART_PosEnrolment[ART_PosEnrolment_columns[0]] - df_ART_PosEnrolment["Total new positive"],
            0
        )
        df_ART_PosEnrolment[ART_PosEnrolment_gap_columns[1]] = np.where(  # -- Step 6.2: ART linkage gap
            df_ART_PosEnrolment[ART_PosEnrolment_columns[1]] != df_ART_PosEnrolment[ART_PosEnrolment_columns[0]],
            df_ART_PosEnrolment[ART_PosEnrolment_columns[1]] - df_ART_PosEnrolment[ART_PosEnrolment_columns[0]],
            0
        )

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # -- Check cache existence
                cached_shape = getattr(process_ART_PosEnrolment_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_PosEnrolment.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    print("Using cached styles")           # -- Confirm cache hit
                    for cluster, style in process_ART_PosEnrolment_gap.cached_styles.items():  # -- Iterate cached styles
                        cached_display_name = f"✔️ Displaying {cluster} {report_name} "  # -- Display message
                        print("-" * len(cached_display_name))  # -- Separator
                        print(cached_display_name)            # -- Message
                        print("-" * len(cached_display_name))  # -- Separator
                        display(style)                        # -- Display cached style
                    return                                 # -- Exit if cache used
                    
        # -- Step 5: Initialize cache
        if not hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # -- Check if cache exists
            process_ART_PosEnrolment_gap.cached_styles = {}  # -- Initialize cache

        # -- Step 7: Identify unique clusters
        cluster_list = pd.Series(df_ART_PosEnrolment['Cluster'].unique())  # -- Extract unique clusters

        # -- Step 8: Process each cluster
        for current_cluster in cluster_list:               # -- Iterate over clusters
            cluster_filtered = df_ART_PosEnrolment[df_ART_PosEnrolment['Cluster'] == current_cluster]  # -- Step 8.1: Filter cluster
            
            ART_PosEnrolment_msg = f"No {current_cluster} {report_name}"  # -- Message for no gaps

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=cluster_filtered,
                msg=ART_PosEnrolment_msg,
                opNonZero=ART_PosEnrolment_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

            if cluster_filtered_gap is None:               # -- Check if no gaps
                if current_cluster in process_ART_PosEnrolment_gap.cached_styles:  # -- Remove from cache
                    del process_ART_PosEnrolment_gap.cached_styles[current_cluster]
                continue                                   # -- Skip cluster

            cluster_filtered_style = (                     # -- Step 8.3: Style DataFrame
                cluster_filtered_gap.style
                .hide(axis='index')
                .map(outlier_red, subset=ART_PosEnrolment_gap_columns)
            )

            process_ART_PosEnrolment_gap.cached_styles[current_cluster] = cluster_filtered_style  # -- Step 8.4: Cache style

            # -- Step 8.5: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # -- Cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # -- Extract report month
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # -- Image file name
            report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # -- Image path
            report_sheet_name = f"{current_cluster}_{report_name}"  # -- Excel sheet name

            # -- Step 9: Create descriptions
            report_description = []                        # -- Initialize descriptions
            if (cluster_filtered_gap[ART_PosEnrolment_gap_columns[0]] != 0).any():  # -- Step 9.1: Enrolment gap desc
                report_description.append(
                    f"Report Name: {ART_PosEnrolment_gap_columns[0]}\n"
                    f"{ART_PosEnrolment_columns_desc[0]}\nshould be equal to {ART_PosEnrolment_columns_desc[2]}\n"
                    f"Note: Where this ART enrolment gap is true, please ignore the outlier."
                )
            if (cluster_filtered_gap[ART_PosEnrolment_gap_columns[1]] != 0).any():  # -- Step 9.2: Linkage gap desc
                report_description.append(
                    f"Report Name: {ART_PosEnrolment_gap_columns[1]}\n"
                    f"{ART_PosEnrolment_columns_desc[1]}\nshould be equal to {ART_PosEnrolment_columns_desc[0]}\n"
                    f"Note: Where this ART linkage gap is true, please ignore the outlier."
                )
            report_description = "\n\n".join(report_description)  # -- Step 9.3: Join descriptions

            # -- Step 10: Export results
            export_df_to_doc_image_excel(                  # -- Export DataFrame
                report_name=report_name_cluster,
                df_shape=cluster_filtered_gap,
                df_style=cluster_filtered_style,
                img_file_name=report_image_name,
                img_file_path=sub_folder2_image_file_msf_outlier,
                doc_description=report_description,
                doc_indicators_to_italicize=ART_PosEnrolment_columns_desc,
                doc_indicators_to_underline=ART_PosEnrolment_gap_columns,
                xlm_file_path=doc_file_msf_outlier_xlsx,
                xlm_sheet_name=report_sheet_name
            )

            if display_output:                             # -- Step 11: Display styled DataFrame
                display(cluster_filtered_style)

        # -- Step 12: Cache overall unfiltered DataFrame shape
        process_ART_PosEnrolment_gap.cached_shape = df_ART_PosEnrolment.shape  # -- Cache shape

    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_PosEnrolment_gap.cached_styles.clear()
        if hasattr(process_ART_PosEnrolment_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_PosEnrolment_gap.cached_shape
        return

### Console

In [13]:
## -- List of function descriptions for display
function_description_name = [
    "Get Data",                                # -- Description for LGA report rate gap
    "Generate Report",                         # -- Description for report generating
    "ANSO Report Rate",                        # -- Description for LGA report rate gap
    "LGA Report Rate",                         # -- Description for facility report rate gap
    "AGYW HTS",                                # -- Description for AGYW HTS gap
    "AGYW Pos",                                # -- Description for AGYW positive gap
    "AGYW Pos Linkage",                        # -- Description for AGYW positive linkage gap 
    "AGYW TB Screening",                       # -- Description for AGYW TB screening gap
    "ART PosEnrolment"                         # -- Description for ART positive enrolment gap
]

# -- Define constants for UI elements
ui_title = "ANSO IHVN DHIS2 MSF Console"       # -- Report title to be displayed in bold
author = "Reuben Edidiong"                     # -- Author name, kept plain due to terminal italic limitation
version = "msf.vlr v1.0"                       # -- Version identifier for the report
ui_sepperator_line = 150                       # -- Length of separator lines (adjustable; 80 for cleaner look)
bold = "\033[1m"                               # -- ANSI code for bold text
reset = "\033[0m"                              # -- ANSI code to reset formatting

# -- Core components for separators
header = f"{bold}{ui_title}{reset} {f'© {author} {version}':>122}"  # -- Header with bold title, right-aligned copyright
top_line = f"{'-' * ui_sepperator_line}\n"       # -- Top separator line
bottom_line = f"\n{'-' * ui_sepperator_line}"  # -- Bottom separator line
spacing = "\n" * 15                            # -- Empty line gap in ui_separator_clear

# -- Separator definitions
ui_separator_top = f"{header}\n{top_line}"     # -- Top separator: header followed by a single line
ui_separator_bottom = f"{bottom_line}"         # -- Bottom separator: just a single line
ui_separator_clear = (                         # -- Full clear separator: header, top line, spacing, bottom line
    f"{header}\n"
    f"{top_line}"
    f"{spacing}"
    f"{bottom_line}"
)

def run_jupyter_mode():
    """
    Runs an interactive Jupyter interface with buttons to execute report processing functions.
    Group names are displayed horizontally with a dropdown arrow, bold text, and font size increased by 2 points.
    Sub-buttons appear when a group name is clicked.
    
    Args:
        None
    
    Returns:
        None 
    """

    # -- Step 1: Create buttons with descriptive labels
    botton0 = widgets.Button(description=f"{function_description_name[0]}")
    botton1 = widgets.Button(description=f"{function_description_name[1]}")
    botton2 = widgets.Button(description=f"{function_description_name[2]}")
    botton3 = widgets.Button(description=f"{function_description_name[3]}")
    botton4 = widgets.Button(description=f"{function_description_name[4]}")
    botton5 = widgets.Button(description=f"{function_description_name[5]}")
    botton6 = widgets.Button(description=f"{function_description_name[6]}")
    botton7 = widgets.Button(description=f"{function_description_name[7]}")
    botton8 = widgets.Button(description=f"{function_description_name[8]}")
    clear_button = widgets.Button(description="Clear screen")
    output = widgets.Output()

    # -- Step 2: Define button handlers
    def on_botton0_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            fetch_dhis2_data_interactive_jupyter_mode()
            print(ui_separator_bottom)
            
    def on_botton2_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_lga_report_rate_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton3_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_facility_report_rate_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton4_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_HTS_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton5_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton6_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_Positive_Linkage_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton7_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton8_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_PosEnrolment_gap(display_output=True)
            print(ui_separator_bottom)

    def on_clear_button_click(b):
        with output:
            clear_output()
            print(ui_separator_clear)

    def on_botton1_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_lga_report_rate_gap(display_output=False)
            process_facility_report_rate_gap(display_output=False)
            process_AGYW_HTS_gap(display_output=False)
            process_AGYW_Positive_gap(display_output=False)
            process_AGYW_Positive_Linkage_gap(display_output=False)
            process_AGYW_TB_Screening_gap(display_output=False)
            process_ART_PosEnrolment_gap(display_output=False)
            print(ui_separator_bottom)

    # -- Step 3: Link buttons to their handlers
    botton0.on_click(on_botton0_click)
    botton1.on_click(on_botton1_click)
    botton2.on_click(on_botton2_click)
    botton3.on_click(on_botton3_click)
    botton4.on_click(on_botton4_click)
    botton5.on_click(on_botton5_click)
    botton6.on_click(on_botton6_click)
    botton7.on_click(on_botton7_click)
    botton8.on_click(on_botton8_click)
    clear_button.on_click(on_clear_button_click)

    # -- Step 4: Create group buttons with dropdown arrow, bold text, and larger font
    # -- Define a consistent layout for button width
    group_button_layout = widgets.Layout(width='150px')  # You can adjust the width as needed

    # -- Define style for button text
    group_button_style = {'font_weight': 'bold', 'font_size': '12px'}
    
    general_button = widgets.Button(
        description="General Actions ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    report_rate_botton = widgets.Button(
        description="Report Rates ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    lga_button = widgets.Button(
        description="LGA Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    agyw_button = widgets.Button(
        description="AGYW Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    art_button = widgets.Button(
        description="ART Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )

    # -- Define sub-button containers
    general_sub_buttons = widgets.HBox([botton0, botton1, clear_button])
    report_rate_sub_botton = widgets.HBox([botton2, botton3, clear_button])
    agyw_sub_buttons = widgets.HBox([botton4, botton5, botton6, botton7, clear_button])
    art_sub_buttons = widgets.HBox([botton8, clear_button])

    # -- Track the currently open group
    current_open = [None]
    sub_button_area = widgets.VBox([])

    # -- Step 5: Define group button handlers to toggle sub-buttons
    def update_button_descriptions(closed_button, opened_button):
        for btn in [general_button, report_rate_botton, agyw_button, art_button]:
            if btn != opened_button and btn.description.endswith("▲"):
                btn.description = btn.description.replace("▲", "▼")
        if closed_button and closed_button.description.endswith("▲"):
            closed_button.description = closed_button.description.replace("▲", "▼")
        if opened_button and not opened_button.description.endswith("▲"):
            opened_button.description = opened_button.description.replace("▼", "▲")

    def on_general_button_click(b):
        if current_open[0] == general_button:
            sub_button_area.children = []
            update_button_descriptions(general_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [general_sub_buttons]
            update_button_descriptions(current_open[0], general_button)
            current_open[0] = general_button

    def on_report_rate_botton_click(b):
        if current_open[0] == report_rate_botton:
            sub_button_area.children = []
            update_button_descriptions(report_rate_botton, None)
            current_open[0] = None
        else:
            sub_button_area.children = [report_rate_sub_botton]
            update_button_descriptions(current_open[0], report_rate_botton)
            current_open[0] = report_rate_botton

    def on_lga_button_click(b):
        if current_open[0] == lga_button:
            sub_button_area.children = []
            update_button_descriptions(lga_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [lga_sub_buttons]
            update_button_descriptions(current_open[0], lga_button)
            current_open[0] = lga_button

    def on_agyw_button_click(b):
        if current_open[0] == agyw_button:
            sub_button_area.children = []
            update_button_descriptions(agyw_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [agyw_sub_buttons]
            update_button_descriptions(current_open[0], agyw_button)
            current_open[0] = agyw_button

    def on_art_button_click(b):
        if current_open[0] == art_button:
            sub_button_area.children = []
            update_button_descriptions(art_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [art_sub_buttons]
            update_button_descriptions(current_open[0], art_button)
            current_open[0] = art_button

    # -- Step 6: Link group buttons to their handlers
    general_button.on_click(on_general_button_click)
    report_rate_botton.on_click(on_report_rate_botton_click)
    agyw_button.on_click(on_agyw_button_click)
    art_button.on_click(on_art_button_click)

    # -- Step 7: Create a horizontal layout for group buttons
    group_buttons = widgets.HBox([
        general_button,
        report_rate_botton,
        agyw_button,
        art_button
    ], layout=widgets.Layout(
        align_items="flex-start",
        padding="10px"
    ))

    # -- Step 8: Create the main layout
    layout = widgets.VBox([
        group_buttons,
        sub_button_area,
        output
    ], layout=widgets.Layout(
        align_items="flex-start",
        padding="10px"
    ))

    # -- Step 9: Display the interface
    display(layout)

# -- Ensure this is the last cell in your notebook
run_jupyter_mode()